# WILDFIRESAI - DATA PIPELINE (Base Notebook)

## Cell 1 — Setup (one-time dependency install)


In [2]:
%pip install --quiet pystac-client planetary-computer


Note: you may need to restart the kernel to use updated packages.


In [3]:
import importlib
print("pystac-client:", importlib.import_module("pystac_client").__version__)
print("planetary-computer:", importlib.import_module("planetary_computer").__version__)


pystac-client: 0.9.0
planetary-computer: 1.0.0


## Cell 2 – Scientific/Infra Stack & Version Audit
Load the full geospatial + scientific + infra stack once.  
Print versions for reproducibility.


In [4]:
## Cell 2 — Imports & Version Audit (centralized)

# ---- Core Python / Utilities ----
import os, sys
from pathlib import Path
from datetime import date, datetime, timedelta, timezone

# ---- Scientific ----
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import openai

# ---- Geospatial & Remote Sensing ----
import geopandas as gpd
import rasterio
from rasterio.merge import merge as rio_merge      # used to mosaic rasters
from rasterio.windows import Window                # used to window/clip rasters
import xarray as xr
import rioxarray
import pyproj
import shapely
import fiona

# ---- Mapping / Visualization ----
import folium
import contextily as ctx

# ---- STAC / Planetary (for ESA WorldCover) ----
from pystac_client import Client                   # STAC search
import planetary_computer as pc                    # signed asset URLs

# ---- Infra / Project Support ----
from dotenv import load_dotenv
import structlog
import tqdm                                        # module-level import
import pyarrow as pa
import pyarrow.parquet as pq
import yaml                                        # pyyaml
import importlib.metadata as im

# ---- Version Audit ----
print("Python:", sys.version.split()[0])
print("NumPy:", np.__version__, "| Pandas:", pd.__version__, "| Matplotlib:", plt.matplotlib.__version__)
print("Requests:", requests.__version__, "| OpenAI SDK:", getattr(openai, "__version__", "unknown"))
print("GeoPandas:", gpd.__version__, "| Rasterio:", rasterio.__version__, "| Xarray:", xr.__version__)
print("rioxarray:", rioxarray.__version__, "| pyproj:", pyproj.__version__, "| Shapely:", shapely.__version__)
print("Fiona:", fiona.__version__, "| Contextily:", ctx.__version__)
print("pystac-client:", im.version("pystac-client"), "| Planetary Computer module:", pc.__name__)
print("structlog:", getattr(structlog, '__version__', 'ok'), "| pyarrow:", pa.__version__)
print("tqdm:", im.version("tqdm"), "| PyYAML:", yaml.__version__)


Python: 3.13.5
NumPy: 2.3.3 | Pandas: 2.3.2 | Matplotlib: 3.10.6
Requests: 2.32.4 | OpenAI SDK: 1.107.1
GeoPandas: 1.1.1 | Rasterio: 1.4.3 | Xarray: 2025.9.1
rioxarray: 0.19.0 | pyproj: 3.7.2 | Shapely: 2.1.2
Fiona: 1.10.1 | Contextily: 1.6.2
pystac-client: 0.9.0 | Planetary Computer module: planetary_computer
structlog: 25.4.0 | pyarrow: 21.0.0
tqdm: 4.67.1 | PyYAML: 6.0.3


## Cell 3 – Project Header & Global Configuration
Defines project paths, environment variables, logging, and small I/O helpers.  
Ensures reproducibility, consistent data handling, and clean outputs across the pipeline.

In [5]:
# ==== WildfiresAI — Cell 3: Project Header & Global Configuration ====

from __future__ import annotations
from typing import Optional

from datetime import date, timedelta
import os
from pathlib import Path

import pandas as pd

from dotenv import load_dotenv
import structlog
from tqdm import tqdm
import pyarrow as pa

# ---- Project paths (shared) ----
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR = PROJECT_ROOT / "reports"
CONFIG_DIR = PROJECT_ROOT / "configs"

for p in (DATA_DIR, RAW_DIR, PROCESSED_DIR, REPORTS_DIR, CONFIG_DIR):
    p.mkdir(parents=True, exist_ok=True)

# ---- Spatial / temporal defaults ----
SPAIN_BBOX = (-9.5, 35.0, 3.5, 43.9)  # WGS84 (EPSG:4326)

TODAY = date.today()
DAYS_BACK = 7  # default temporal window
DATE_FROM = os.getenv("WF_DATE_FROM", str(TODAY - timedelta(days=DAYS_BACK)))
DATE_TO   = os.getenv("WF_DATE_TO", str(TODAY))

# ---- Secrets (credentials if available) ----
load_dotenv()  # Load from .env at project root
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
FIRMS_TOKEN    = os.getenv("FIRMS_TOKEN", "")   # optional (Bearer endpoints)
FIRMS_MAP_KEY  = os.getenv("FIRMS_MAP_KEY", "") # required for FIRMS CSV API if used
WF_REGION      = os.getenv("WF_REGION", "ES")

# ---- Lightweight logging (structured) ----
structlog.configure(
    wrapper_class=structlog.make_filtering_bound_logger(20)  # INFO
)
log = structlog.get_logger("wildfiresai").bind(region=WF_REGION)
log.info("Init config",
         root=PROJECT_ROOT,
         raw=RAW_DIR,
         processed=PROCESSED_DIR,
         reports=REPORTS_DIR)

# ---- Small helpers (I/O, display, masking) ----
def mask_key(key: str, n: int = 6) -> str:
    """Mask sensitive keys, keeping only first n chars."""
    return key[:n] + "…" if key else "None"

def save_df(df: pd.DataFrame, path: Path) -> None:
    """Save DataFrame to CSV at the given path (parents created if needed)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False)
    log.info("Saved CSV", path=str(path))

def save_parquet(df: pd.DataFrame, path: Path) -> None:
    """Save DataFrame to Parquet at the given path."""
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(path, index=False)
    log.info("Saved Parquet", path=str(path))

def preview(df: pd.DataFrame, n: int = 6) -> pd.DataFrame:
    """Return the first n rows for consistent preview across the notebook."""
    return df.head(n)

# ---- Echo effective configuration (concise) ----
print("Paths:", f"raw={RAW_DIR.name}, processed={PROCESSED_DIR.name}, reports={REPORTS_DIR.name}")
print("Dates:", f"{DATE_FROM} → {DATE_TO}, Window={DAYS_BACK} days")
print("Region:", WF_REGION, "| BBOX:", SPAIN_BBOX)
print("Secrets:", f"OpenAI={mask_key(OPENAI_API_KEY)}, "
                 f"FIRMS_TOKEN={mask_key(FIRMS_TOKEN)}, "
                 f"MAP_KEY={mask_key(FIRMS_MAP_KEY)}")


2025-09-30 20:22:44 [info     ] Init config                    processed=PosixPath('/Users/evareysanchez/WildfiresAI/data/processed') raw=PosixPath('/Users/evareysanchez/WildfiresAI/data/raw') region=ES reports=PosixPath('/Users/evareysanchez/WildfiresAI/reports') root=PosixPath('/Users/evareysanchez/WildfiresAI')
Paths: raw=raw, processed=processed, reports=reports
Dates: 2025-09-23 → 2025-09-30, Window=7 days
Region: ES | BBOX: (-9.5, 35.0, 3.5, 43.9)
Secrets: OpenAI=sk-pro…, FIRMS_TOKEN=eyJ0eX…, MAP_KEY=27f8d7…


## Cell 4 – Unified Source Configuration
Defines all external data sources for fires, weather, climate, hydrology, and terrain.  
Centralized registry ensures consistency, reproducibility, and easy updates across the pipeline.

In [6]:
# ==== WildfiresAI — Cell 4: Unified Source Configuration ====
# Sources are declared here and controlled via environment variables when required.

# ---- Fire sources ----
SRC_FIRES = {
    "FIRMS": {
        "enabled": True,
        "endpoint": os.getenv("FIRMS_CSV_URL", ""),
        "token": os.getenv("FIRMS_TOKEN", ""),
        "map_key": os.getenv("FIRMS_MAP_KEY", ""),
        "notes": "Global near real-time fire detections (VIIRS/MODIS)."
    },
    "EFFIS": {
        "enabled": True,
        "wfs_url": os.getenv("EFFIS_WFS_URL", ""),
        "notes": "Europe fire perimeters and burned area (via Copernicus Emergency)."
    },
    "NIFC": {
        "enabled": True,
        "featureserver_url": os.getenv("NIFC_FEATURESERVER_URL", ""),
        "notes": "United States wildfire perimeters/incidents (NIFC FeatureServer)."
    },
}

# ---- Weather & climate (via Open-Meteo) ----
SRC_WEATHER = {
    "OPEN_METEO_FORECAST": {
        "enabled": True,
        "base_url": "https://api.open-meteo.com/v1/forecast",
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "windspeed_10m",
            "winddirection_10m", "windgusts_10m", "dew_point_2m",
            "precipitation", "surface_pressure"
        ],
        "models": os.getenv("OM_MODELS", "gfs_seamless"),
        "notes": "Global short-term forecast (GFS-backed when models=gfs_*)."
    },
    "OPEN_METEO_ARCHIVE": {
        "enabled": True,
        "base_url": "https://archive-api.open-meteo.com/v1/archive",
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "windspeed_10m",
            "winddirection_10m", "dew_point_2m", "precipitation",
            "surface_pressure"
        ],
        "notes": "Historical archive (ERA5-backed hourly data)."
    },
    "ERA5_ALIAS": {"enabled": True, "notes": "Handled via OPEN_METEO_ARCHIVE."},
    "GFS_ALIAS": {"enabled": True, "notes": "Handled via OPEN_METEO_FORECAST."},
}

# ---- Hydrology / drought ----
SRC_HYDRO = {
    "GRACE": {
        "enabled": True,
        "url": os.getenv("GRACE_URL", ""),
        "earthdata_user": os.getenv("EARTHDATA_USERNAME", ""),
        "earthdata_pass": os.getenv("EARTHDATA_PASSWORD", ""),
        "notes": "Groundwater/drought indicators (NASA GRACE)."
    }
}

# ---- Terrain & land cover ----
SRC_TERRAIN = {
    "GLOBAL_BASELINE": {
        "enabled": True,
        "dem": "Copernicus DEM GLO-30",
        "landcover": "ESA WorldCover 10m",
        "notes": "Global baseline terrain and land cover."
    },
    "EUROPE": {
        "enabled": True,
        "dem": "EU-DEM 25m",
        "landcover": "CORINE Land Cover",
        "notes": "European terrain and land cover products."
    },
    "USA": {
        "enabled": True,
        "dem": "USGS 3DEP 1m",
        "notes": "High-resolution DEM for the United States."
    },
}

# ---- Region windows (BBOX) ----
REGION_CFG = {
    "ES": {"bbox": (-9.5, 35.0, 3.5, 43.9)},        # Spain
    "US": {"bbox": (-125.0, 24.4, -66.9, 49.4)},    # Conterminous US
    "EU": {"bbox": (-25.0, 34.0, 45.0, 72.0)},      # Europe
}
ACTIVE_BBOX = REGION_CFG.get(WF_REGION, REGION_CFG["ES"])["bbox"]

# ---- Central registry ----
CFG_SOURCES = {
    "region": WF_REGION,
    "bbox": ACTIVE_BBOX,
    "window": {"from": DATE_FROM, "to": DATE_TO},
    "fires": SRC_FIRES,
    "weather": SRC_WEATHER,
    "hydrology": SRC_HYDRO,
    "terrain": SRC_TERRAIN,
}

# ---- Summary table ----
def summarize_sources(cfg: dict) -> pd.DataFrame:
    rows = []
    for group, entries in cfg.items():
        if isinstance(entries, dict):
            for name, meta in entries.items():
                if isinstance(meta, dict) and "enabled" in meta:
                    rows.append({
                        "group": group,
                        "name": name,
                        "enabled": meta["enabled"],
                        "detail": meta.get("notes", "")
                    })
    return pd.DataFrame(rows, columns=["group","name","enabled","detail"])

print("Active region:", CFG_SOURCES["region"], "| BBOX:", CFG_SOURCES["bbox"])
display(summarize_sources(CFG_SOURCES))

    

Active region: ES | BBOX: (-9.5, 35.0, 3.5, 43.9)


,group,name,enabled,detail
0,fires,FIRMS,True,Global near real-time fire detections (VIIRS/M...
1,fires,EFFIS,True,Europe fire perimeters and burned area (via Co...
2,fires,NIFC,True,United States wildfire perimeters/incidents (N...
3,weather,OPEN_METEO_FORECAST,True,Global short-term forecast (GFS-backed when mo...
4,weather,OPEN_METEO_ARCHIVE,True,Historical archive (ERA5-backed hourly data).
5,weather,ERA5_ALIAS,True,Handled via OPEN_METEO_ARCHIVE.
6,weather,GFS_ALIAS,True,Handled via OPEN_METEO_FORECAST.
7,hydrology,GRACE,True,Groundwater/drought indicators (NASA GRACE).
8,terrain,GLOBAL_BASELINE,True,Global baseline terrain and land cover.
9,terrain,EUROPE,True,European terrain and land cover products.


## Cell 5 – IngestAgent (Fires, Weather, Hydrology)
Downloads and caches raw datasets: FIRMS, EFFIS (WFS), NIFC (FeatureServer), Open-Meteo (forecast/archive), and GRACE.
Outputs are stored under `data/raw/<source>/...` with a concise ingest report.


In [7]:
## Cell 5 — IngestAgent (FIRMS + Weather + hooks EFFIS/NIFC)

import os, json, time
from pathlib import Path
from datetime import datetime, timezone
from typing import Optional, Dict, Any, Tuple

import pandas as pd
import requests
import structlog
from dotenv import load_dotenv

# --- env & paths
PROJECT_ROOT = Path.cwd()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
REPORTS_DIR = PROJECT_ROOT / "reports"
RAW_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

load_dotenv()  # read .env once
log = structlog.get_logger("ingest").bind(region=os.getenv("WF_REGION", "ES"))

# --- config (from Cell 3 defaults)
SPAIN_BBOX = globals().get("SPAIN_BBOX", (-9.5, 35.0, 3.5, 43.9))
DATE_FROM = globals().get("DATE_FROM")
DATE_TO   = globals().get("DATE_TO")

# --- endpoints & keys
FIRMS_MAP_KEY  = os.getenv("FIRMS_MAP_KEY")
FIRMS_TOKEN    = os.getenv("FIRMS_TOKEN")          # optional (Bearer)
FIRMS_CSV_URL  = os.getenv("FIRMS_CSV_URL")        # required for CSV mode

EFFIS_WFS_URL  = os.getenv("EFFIS_WFS_URL")        # optional
NIFC_FS_URL    = os.getenv("NIFC_FS_URL")          # optional

def _bbox_dict(bbox: Tuple[float,float,float,float]) -> Dict[str, float]:
    w, s, e, n = bbox
    return {"west": w, "south": s, "east": e, "north": n}

def _save_json(obj: Any, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

artifacts = []

# --- 5.1 FIRMS (CSV mode with MAP_KEY) ---------------------------------
try:
    if not FIRMS_CSV_URL:
        log.warning("firms_endpoint_missing", hint="Set FIRMS_CSV_URL in env")
    else:
        from datetime import timezone
        dt_from = datetime.fromisoformat(str(DATE_FROM))
        dt_to   = datetime.fromisoformat(str(DATE_TO))
        window_days = max(1, min((dt_to - dt_from).days or 1, 10))

        # Compose URL directly with bbox and day inside the path
        w, s, e, n = SPAIN_BBOX
        url = f"{FIRMS_CSV_URL}/{w},{s},{e},{n}/{window_days}"

        r = requests.get(url, timeout=90)
        r.raise_for_status()

        ts = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
        out_csv = RAW_DIR / f"firms_viirs_snpp_{DATE_FROM}_{DATE_TO}_{ts}.csv"
        out_csv.write_text(r.text, encoding="utf-8")

        log.info("firms_csv_downloaded", path=str(out_csv))
        artifacts.append(("firms", str(out_csv)))

except Exception as e:
    log.warning("firms_download_failed", err=str(e))




# --- 5.2 Open-Meteo (archive + forecast) --------------------------------
try:
    W, S, E, N = SPAIN_BBOX
    lat_c, lon_c = (S + N) / 2.0, (W + E) / 2.0

    ts = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
    hourly_vars = [
        "temperature_2m", "relative_humidity_2m", "dew_point_2m",
        "windspeed_10m", "windgusts_10m", "winddirection_10m",
        "precipitation", "surface_pressure"
    ]
    params_common = {
        "latitude": round(lat_c, 4),
        "longitude": round(lon_c, 4),
        "start_date": str(DATE_FROM),
        "end_date": str(DATE_TO),
        "hourly": ",".join(hourly_vars),
        "timezone": "UTC"
    }

    # Archive (ERA5-backed)
    om_archive = os.getenv("OM_ARCHIVE", "https://archive-api.open-meteo.com/v1/archive")
    r = requests.get(om_archive, params=params_common, timeout=90)
    r.raise_for_status()
    out_archive = RAW_DIR / "openmeteo_archive" / f"archive_{lat_c:.3f}_{lon_c:.3f}_{DATE_FROM}_{DATE_TO}_{ts}.json"
    out_archive.parent.mkdir(parents=True, exist_ok=True)
    _save_json(r.json(), out_archive)
    log.info("openmeteo_downloaded", label="archive", path=str(out_archive))
    artifacts.append(("om_archive", str(out_archive)))

    # Forecast (GFS-backed)
    om_forecast = os.getenv("OM_FORECAST", "https://api.open-meteo.com/v1/forecast")
    r2 = requests.get(om_forecast, params={**params_common, "models": "gfs_seamless"}, timeout=90)
    r2.raise_for_status()
    out_forecast = RAW_DIR / "openmeteo_forecast" / f"forecast_{lat_c:.3f}_{lon_c:.3f}_{DATE_FROM}_{DATE_TO}_{ts}.json"
    out_forecast.parent.mkdir(parents=True, exist_ok=True)
    _save_json(r2.json(), out_forecast)
    log.info("openmeteo_downloaded", label="forecast", path=str(out_forecast))
    artifacts.append(("om_forecast", str(out_forecast)))

except Exception as e:
    log.warning("openmeteo_failed", err=str(e))

# --- 5.3 EFFIS (optional; WFS) -----------------------------------------
try:
    effis_url = os.getenv("EFFIS_WFS_URL")
    effis_lyr = os.getenv("EFFIS_TYPENAME")
    if effis_url and effis_lyr:
        # Ejemplo de GetFeature WFS → GeoJSON
        w, s, e, n = SPAIN_BBOX
        params = {
            "service": "WFS",
            "request": "GetFeature",
            "version": "2.0.0",
            "typeNames": effis_lyr,             
            "srsName": "EPSG:4326",
            "bbox": f"{s},{w},{n},{e},EPSG:4326",
            "outputFormat": "application/json"
        }
        r = requests.get(effis_url, params=params, timeout=120)
        r.raise_for_status()
        out_effis = RAW_DIR / "effis" / f"perimeters_{DATE_FROM}_{DATE_TO}_{ts}.geojson"
        out_effis.parent.mkdir(parents=True, exist_ok=True)
        _save_json(r.json(), out_effis)
        log.info("effis_geojson_downloaded", path=str(out_effis))
        artifacts.append(("effis_perimeters", str(out_effis)))
    # Sin URL/typename → silencio (no “skipped”) hasta que tengas acceso oficial.

except Exception as e:
    log.warning("effis_failed", err=str(e))


# --- 5.4 NIFC (USA perimeters; ArcGIS FeatureServer → GeoJSON) ---------
try:
    # Usamos un valor por defecto robusto si la var no está (vista "Current").
    nifc_url = os.getenv(
        "NIFC_FS_URL",
        "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/WFIGS_Interagency_Perimeters_Current/FeatureServer/0"
    )

    if nifc_url:
       
        w, s, e, n = SPAIN_BBOX  # puedes usar otra bbox según región
        geometry = {
            "xmin": w, "ymin": s, "xmax": e, "ymax": n,
            "spatialReference": {"wkid": 4326}
        }
        q = {
            "where": "1=1",
            "geometryType": "esriGeometryEnvelope",
            "geometry": json.dumps(geometry),
            "inSR": 4326,
            "spatialRel": "esriSpatialRelIntersects",
            "outFields": "*",
            "returnGeometry": "true",
            "f": "geojson"  # salida directa GeoJSON
        }
        r = requests.get(f"{nifc_url}/query", params=q, timeout=120)
        r.raise_for_status()
        gj = r.json()

        out_nifc = RAW_DIR / "nifc" / f"perimeters_{DATE_FROM}_{DATE_TO}_{ts}.geojson"
        out_nifc.parent.mkdir(parents=True, exist_ok=True)
        _save_json(gj, out_nifc)
        log.info("nifc_geojson_downloaded", path=str(out_nifc))
        artifacts.append(("nifc_perimeters", str(out_nifc)))
   

except Exception as e:
    log.warning("nifc_failed", err=str(e))


# --- 5.5 Report ---------------------------------------------------------
ingest_report = REPORTS_DIR / f"ingest_report_{os.getenv('WF_REGION','ES')}_{DATE_FROM}_{DATE_TO}.json"
_save_json({"artifacts": artifacts}, ingest_report)
print(f"Ingest report: {ingest_report}")

# Display a compact table
pd.DataFrame(artifacts, columns=["source","artifact"])





2025-09-30 20:22:51 [info     ] firms_csv_downloaded           path=/Users/evareysanchez/WildfiresAI/data/raw/firms_viirs_snpp_2025-09-23_2025-09-30_20250930T182251Z.csv region=ES
2025-09-30 20:22:51 [info     ] openmeteo_downloaded           label=archive path=/Users/evareysanchez/WildfiresAI/data/raw/openmeteo_archive/archive_39.450_-3.000_2025-09-23_2025-09-30_20250930T182251Z.json region=ES
2025-09-30 20:22:51 [info     ] openmeteo_downloaded           label=forecast path=/Users/evareysanchez/WildfiresAI/data/raw/openmeteo_forecast/forecast_39.450_-3.000_2025-09-23_2025-09-30_20250930T182251Z.json region=ES
2025-09-30 20:22:52 [info     ] nifc_geojson_downloaded        path=/Users/evareysanchez/WildfiresAI/data/raw/nifc/perimeters_2025-09-23_2025-09-30_20250930T182251Z.geojson region=ES
Ingest report: /Users/evareysanchez/WildfiresAI/reports/ingest_report_ES_2025-09-23_2025-09-30.json


,source,artifact
0,firms,/Users/evareysanchez/WildfiresAI/data/raw/firm...
1,om_archive,/Users/evareysanchez/WildfiresAI/data/raw/open...
2,om_forecast,/Users/evareysanchez/WildfiresAI/data/raw/open...
3,nifc_perimeters,/Users/evareysanchez/WildfiresAI/data/raw/nifc...


## Cell 6 – CleanerAgent (Robust Normalization)
Normalize raw inputs into consistent, typed tables ready for terrain enrichment and spatiotemporal joins.

**Outputs**
- `data/processed/fires_clean.parquet`
- `data/processed/effis_clean.parquet` (if available)
- `data/processed/nifc_clean.parquet` (if available)
- `data/processed/weather_points.parquet`
- `reports/clean_report.json`

In [8]:
## Cell 6 — CleanerAgent (FIRMS + Open-Meteo + NIFC; robust validation & sampling)

import os, json
from pathlib import Path
import pandas as pd

# Paths from earlier cells
PROC_DIR = PROJECT_ROOT / "data" / "processed"
RAW_DIR = PROJECT_ROOT / "data" / "raw"
REPORTS_DIR = PROJECT_ROOT / "reports"
for p in (PROC_DIR, RAW_DIR, REPORTS_DIR):
    p.mkdir(parents=True, exist_ok=True)

clean_artifacts: list[tuple[str,str]] = []

# ---------------- Helpers ----------------

def _save_report(obj: dict, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)

def _sniff_sep(path: Path) -> str:
    # Delimiter sniffer between comma and semicolon
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    return "," if sample.count(",") >= sample.count(";") else ";"

def _find_col(colmap_lower: dict[str,str], candidates: list[str]) -> str | None:
    # Return original-case column name for the first lower-cased match
    for c in candidates:
        if c in colmap_lower:
            return colmap_lower[c]
    return None

def _build_datetime(df: pd.DataFrame, date_col: str | None, time_col: str | None, dt_col: str | None) -> pd.Series:
    # Build timezone-aware timestamp (UTC). If missing parts, coerce to NaT.
    if dt_col and dt_col in df.columns:
        return pd.to_datetime(df[dt_col], errors="coerce", utc=True)
    if date_col and time_col and date_col in df.columns and time_col in df.columns:
        t = df[time_col].astype(str).str.zfill(4)
        return pd.to_datetime(
            df[date_col].astype(str) + " " + t.str[:2] + ":" + t.str[2:4] + ":00",
            errors="coerce", utc=True
        )
    if date_col and date_col in df.columns:
        return pd.to_datetime(df[date_col], errors="coerce", utc=True)
    return pd.to_datetime(pd.NaT)

# ---------------- 6.1 Clean FIRMS (CSV) ----------------

try:
    firms_raw = sorted(RAW_DIR.glob("firms_viirs_snpp_*.csv"))
    if not firms_raw:
        log.warning("clean_firms_missing", hint="No FIRMS CSV found in data/raw/")
    else:
        latest = firms_raw[-1]
        sep = _sniff_sep(latest)

        df_firms = pd.read_csv(latest, sep=sep, engine="python", dtype={"acq_time": "string"})
        df_firms.columns = [c.strip() for c in df_firms.columns]
        colmap_lower = {c.lower(): c for c in df_firms.columns}

        lat_col = _find_col(colmap_lower, ["latitude", "lat", "y", "latitud"])
        lon_col = _find_col(colmap_lower, ["longitude", "lon", "x", "longitud", "long"])
        if not lat_col or not lon_col:
            raise KeyError("FIRMS CSV lacks latitude/longitude columns (tried latitude|lat|y and longitude|lon|x variants)")

        dt_col   = _find_col(colmap_lower, ["acq_datetime", "datetime", "time_utc"])
        date_col = _find_col(colmap_lower, ["acq_date", "date", "fecha"])
        time_col = _find_col(colmap_lower, ["acq_time", "time", "hora"])

        out = pd.DataFrame({
            "lat":  pd.to_numeric(df_firms[lat_col], errors="coerce"),
            "lon":  pd.to_numeric(df_firms[lon_col], errors="coerce"),
            "datetime": _build_datetime(df_firms, date_col, time_col, dt_col),
            "brightness": pd.to_numeric(df_firms[_find_col(colmap_lower, ["brightness", "bright_ti4", "bright_ti5"])], errors="coerce") if _find_col(colmap_lower, ["brightness","bright_ti4","bright_ti5"]) else pd.NA,
            "confidence": pd.to_numeric(df_firms[_find_col(colmap_lower, ["confidence"])], errors="coerce") if _find_col(colmap_lower, ["confidence"]) else pd.NA,
            "frp": pd.to_numeric(df_firms[_find_col(colmap_lower, ["frp"])], errors="coerce") if _find_col(colmap_lower, ["frp"]) else pd.NA,
            "satellite": df_firms[_find_col(colmap_lower, ["satellite"])] if _find_col(colmap_lower, ["satellite"]) else pd.NA,
            "source": "FIRMS"
        })

        out = out.dropna(subset=["lat", "lon"]).drop_duplicates(subset=["datetime","lat","lon"]).reset_index(drop=True)

        ACTIVE_BBOX = globals().get("ACTIVE_BBOX", globals().get("SPAIN_BBOX"))
        if ACTIVE_BBOX is not None:
            w, s, e, n = ACTIVE_BBOX
            out = out[(out["lon"].between(w, e)) & (out["lat"].between(s, n))].copy()

        if "DATE_FROM" in globals() and "DATE_TO" in globals() and "datetime" in out.columns:
            out = out[(out["datetime"] >= pd.to_datetime(DATE_FROM, utc=True)) &
                      (out["datetime"] <= pd.to_datetime(DATE_TO,   utc=True))].copy()

        region = os.getenv("WF_REGION", "ES")
        out_firms = PROC_DIR / f"firms_clean_{region}_{DATE_FROM}_{DATE_TO}.csv"
        out.to_csv(out_firms, index=False)
        log.info("firms_clean_saved", rows=len(out), path=str(out_firms))

        sample_firms = REPORTS_DIR / f"sample_firms_{region}_{DATE_FROM}_{DATE_TO}.json"
        out.head(10).to_json(sample_firms, orient="records", indent=2)
        clean_artifacts.append(("fires", str(out_firms)))

except Exception as e:
    log.warning("clean_firms_fail", err=str(e))

# ---------------- 6.2 Clean Open-Meteo (archive JSON → parquet) ----------------

try:
    wx_archives = sorted((RAW_DIR / "openmeteo_archive").glob("archive_*.json"))
    if not wx_archives:
        log.warning("clean_weather_missing", hint="No Open-Meteo archive found")
    else:
        js = json.load(open(wx_archives[-1], encoding="utf-8"))
        hourly = js.get("hourly", {})
        df_wx = pd.DataFrame(hourly)

        if "time" in df_wx.columns:
            df_wx["datetime"] = pd.to_datetime(df_wx["time"], errors="coerce", utc=True)

        out_wx = PROC_DIR / "weather_points.parquet"
        df_wx.to_parquet(out_wx, index=False)
        log.info("weather_points_saved", rows=len(df_wx), path=str(out_wx))

        sample_wx = REPORTS_DIR / f"sample_weather_{os.getenv('WF_REGION','ES')}_{DATE_FROM}_{DATE_TO}.json"
        df_wx.head(10).to_json(sample_wx, orient="records", indent=2)
        clean_artifacts.append(("weather_points", str(out_wx)))
except Exception as e:
    log.warning("clean_weather_fail", err=str(e))

# ---------------- 6.3 Clean NIFC (GeoJSON perimeters → centroid points) ----------------
try:
    nifc_files = sorted((RAW_DIR / "nifc").glob("perimeters_*.geojson"))
    if nifc_files:
        latest_nifc = nifc_files[-1]

        # Prefer geopandas if available; otherwise use a robust JSON fallback
        try:
            import geopandas as gpd  # guarded import (no duplicate global imports)
            gdf = gpd.read_file(latest_nifc)

            # Ensure we start in geographic CRS for bbox filtering and then project to metric CRS
            try:
                gdf = gdf.to_crs(4326)
            except Exception:
                pass

            # Compute centroids in a metric CRS for accuracy, then convert back to WGS84
            try:
                gdf_3857 = gdf.to_crs(3857)
                cent_m = gdf_3857.geometry.centroid
                cent_wgs84 = cent_m.to_crs(4326)
                df_nifc = pd.DataFrame({"lon": cent_wgs84.x, "lat": cent_wgs84.y})
            except Exception:
                # Fallback if reprojection fails for some geometries
                cent_wgs84 = gdf.to_crs(4326).geometry.centroid
                df_nifc = pd.DataFrame({"lon": cent_wgs84.x, "lat": cent_wgs84.y})

            # Try to extract a perimeter datetime from attributes if present
            props = gdf.drop(columns=["geometry"], errors="ignore")
            time_cols = [c for c in props.columns if any(k in c.lower() for k in ["date", "time", "datetime", "perimeter"])]
            if time_cols:
                df_nifc["datetime"] = pd.to_datetime(props[time_cols[0]], errors="coerce", utc=True)
            else:
                df_nifc["datetime"] = pd.NaT

            df_nifc["source"] = "NIFC"

        except Exception:
            # JSON fallback: compute crude centroid (mean of ring vertices) per feature
            js = json.load(open(latest_nifc, encoding="utf-8"))
            feats = js.get("features", [])
            rows = []
            for f in feats:
                geom = f.get("geometry", {})
                coords = []
                if geom.get("type") == "Polygon":
                    for ring in geom.get("coordinates", []):
                        coords.extend(ring)
                elif geom.get("type") == "MultiPolygon":
                    for poly in geom.get("coordinates", []):
                        for ring in poly:
                            coords.extend(ring)
                if coords:
                    xs = [float(x) for x, y in coords]
                    ys = [float(y) for x, y in coords]
                    lon = sum(xs) / len(xs)
                    lat = sum(ys) / len(ys)
                    props = f.get("properties", {})
                    tfield = next((k for k in props.keys() if any(s in k.lower() for s in ["date","time","datetime","perimeter"])), None)
                    dt = pd.to_datetime(props.get(tfield), errors="coerce", utc=True) if tfield else pd.NaT
                    rows.append({"lon": lon, "lat": lat, "datetime": dt, "source": "NIFC"})
            df_nifc = pd.DataFrame(rows)

        # Optional spatial and temporal filters based on globals
        ACTIVE_BBOX = globals().get("ACTIVE_BBOX", globals().get("SPAIN_BBOX"))
        if ACTIVE_BBOX is not None and not df_nifc.empty:
            w, s, e, n = ACTIVE_BBOX
            df_nifc = df_nifc[(df_nifc["lon"].between(w, e)) & (df_nifc["lat"].between(s, n))].copy()

        if "DATE_FROM" in globals() and "DATE_TO" in globals() and not df_nifc.empty and "datetime" in df_nifc.columns:
            df_nifc = df_nifc[(df_nifc["datetime"].isna()) |
                              ((df_nifc["datetime"] >= pd.to_datetime(DATE_FROM, utc=True)) &
                               (df_nifc["datetime"] <= pd.to_datetime(DATE_TO,   utc=True)))]

        out_nifc = PROC_DIR / f"nifc_clean_{os.getenv('WF_REGION','ES')}_{DATE_FROM}_{DATE_TO}.parquet"
        if not df_nifc.empty:
            df_nifc.to_parquet(out_nifc, index=False)
            log.info("nifc_clean_saved", rows=len(df_nifc), path=str(out_nifc))
            clean_artifacts.append(("nifc_clean", str(out_nifc)))
        else:
            log.info("nifc_clean_empty")
    # If no NIFC file present, keep silent (clean pipeline)

except Exception as e:
    log.warning("clean_nifc_fail", err=str(e))

# ---------------- 6.4 Report ----------------

clean_report = REPORTS_DIR / f"clean_report_{os.getenv('WF_REGION','ES')}_{DATE_FROM}_{DATE_TO}.json"
_save_report({"artifacts": clean_artifacts}, clean_report)
print(f"Clean report: {clean_report}")

pd.DataFrame(clean_artifacts, columns=["artifact","value"])


2025-09-30 20:22:52 [info     ] firms_clean_saved              path=/Users/evareysanchez/WildfiresAI/data/processed/firms_clean_ES_2025-09-23_2025-09-30.csv region=ES rows=737
2025-09-30 20:22:52 [info     ] weather_points_saved           path=/Users/evareysanchez/WildfiresAI/data/processed/weather_points.parquet region=ES rows=192
2025-09-30 20:22:52 [info     ] nifc_clean_empty               region=ES
Clean report: /Users/evareysanchez/WildfiresAI/reports/clean_report_ES_2025-09-23_2025-09-30.json


,artifact,value
0,fires,/Users/evareysanchez/WildfiresAI/data/processe...
1,weather_points,/Users/evareysanchez/WildfiresAI/data/processe...


## Cell 7 — GeoTerrainAgent (terrain enrichment with auto-fetch)
Enrich fire detections with terrain attributes: elevation, slope, and land cover.
- DEM: OpenTopography (requires OPENTOPO_API_KEY in .env)
- Land cover: ESA WorldCover (via Planetary Computer STAC, fallback years)
 Output: terrain-enriched fires saved to Parquet.


In [9]:
## Cell 7 — GeoTerrainAgent (terrain enrichment with auto-fetch, memory-safe)

from pathlib import Path
import os, json, math
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.transform import Affine
import structlog

log = structlog.get_logger("geoterrain").bind(region=os.getenv("WF_REGION", "ES"))

# --- config / paths
REGION = os.getenv("WF_REGION", "ES")
W, S, E, N = globals().get("ACTIVE_BBOX", globals().get("SPAIN_BBOX", (-9.5, 35.0, 3.5, 43.9)))

TERRAIN_DIR = PROJECT_ROOT / "data" / "terrain"
TERRAIN_DIR.mkdir(parents=True, exist_ok=True)
DEM_TIF = TERRAIN_DIR / "copernicus_dem.tif"
LC_TIF  = TERRAIN_DIR / "esa_worldcover_clip.tif"

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
fires_clean_path   = PROCESSED_DIR / f"firms_clean_{REGION}_{DATE_FROM}_{DATE_TO}.csv"
fires_terrain_out  = PROCESSED_DIR / f"fires_terrain_{REGION}_{DATE_FROM}_{DATE_TO}.parquet"

# ----------------------- helpers: asset download/build -----------------------

def _download_dem_opentopo(w: float, s: float, e: float, n: float, out_path: Path, demtype="SRTMGL3") -> None:
    """
    Fetch DEM GeoTIFF clipped to bbox using OpenTopography Global DEM API (SRTMGL3).
    Requires OPENTOPO_API_KEY in .env.
    """
    import requests
    api_key = os.getenv("OPENTOPO_API_KEY", "")
    url = "https://portal.opentopography.org/API/globaldem"
    params = {
        "demtype": demtype,
        "south": s, "north": n, "west": w, "east": e,
        "outputFormat": "GTiff",
        "API_Key": api_key
    }
    r = requests.get(url, params=params, timeout=600); r.raise_for_status()
    try:
        js = r.json()
        tif_url = None
        if isinstance(js, dict) and "result" in js and "links" in js["result"]:
            for lnk in js["result"]["links"]:
                href = lnk.get("href", "")
                if href.endswith(".tif"):
                    tif_url = href
                    break
        if not tif_url:
            raise RuntimeError("No GeoTIFF link found in OpenTopography response.")
        r2 = requests.get(tif_url, timeout=1200); r2.raise_for_status()
        out_path.write_bytes(r2.content)
        log.info("dem_downloaded", src=tif_url, path=str(out_path))
    except Exception:
        out_path.write_bytes(r.content)
        log.info("dem_downloaded_direct", bytes=len(r.content), path=str(out_path))

def _build_worldcover_from_stac(w: float, s: float, e: float, n: float, out_path: Path) -> None:
    """
    Query ESA WorldCover via Planetary Computer STAC, clip to bbox, and save to GeoTIFF.
    Reads only the bounding window instead of entire mosaics (memory-safe).
    """
    from pystac_client import Client
    import planetary_computer as pc
    import rasterio
    from rasterio.windows import from_bounds

    stac = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    bbox = [w, s, e, n]
    years = [2021, 2020]
    items = []
    last_err = None

    for yr in years:
        try:
            search = stac.search(
                collections=["esa-worldcover"],
                bbox=bbox,
                datetime=f"{yr}-01-01/{yr}-12-31"
            )
            items = list(search.items())
            if items:
                log.info("worldcover_found", year=yr, count=len(items))
                break
        except Exception as ex:
            last_err = ex
            continue

    if not items:
        raise RuntimeError(f"No WorldCover found for bbox in years={years}. Last error: {last_err}")

    for it in items:
        asset = it.assets.get("map") or it.assets.get("Map")
        if not asset:
            continue
        signed_href = pc.sign(asset.href)
        with rasterio.open(signed_href) as src:
            window = from_bounds(w, s, e, n, src.transform)
            data = src.read(1, window=window)
            transform = src.window_transform(window)

            profile = src.profile.copy()
            profile.update({
                "driver": "GTiff",
                "height": data.shape[0],
                "width": data.shape[1],
                "transform": transform,
                "count": 1,
                "dtype": data.dtype,
                "compress": "lzw"
            })

            out_path.parent.mkdir(parents=True, exist_ok=True)
            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(data, 1)
            log.info("landcover_clipped", path=str(out_path), shape=data.shape)
            return

    raise RuntimeError("WorldCover items found but no usable 'map' assets.")

# ----------------------- ensure assets exist -----------------------

try:
    if not DEM_TIF.exists() or DEM_TIF.stat().st_size < 100_000:
        _download_dem_opentopo(W, S, E, N, DEM_TIF)
    else:
        log.info("dem_present", path=str(DEM_TIF), size=DEM_TIF.stat().st_size)
except Exception as e:
    log.warning("dem_unavailable", err=str(e))

try:
    if not LC_TIF.exists() or LC_TIF.stat().st_size < 100_000:
        _build_worldcover_from_stac(W, S, E, N, LC_TIF)
    else:
        log.info("landcover_present", path=str(LC_TIF), size=LC_TIF.stat().st_size)
except Exception as e:
    log.warning("landcover_unavailable", err=str(e))

# ----------------------- load fires & sampling -----------------------

try:
    df_fires = pd.read_csv(fires_clean_path)
    gdf_fires = gpd.GeoDataFrame(df_fires, geometry=gpd.points_from_xy(df_fires.lon, df_fires.lat), crs="EPSG:4326")
except Exception as e:
    log.error("fires_load_failed", err=str(e))
    gdf_fires = gpd.GeoDataFrame(columns=["lat","lon","geometry"], crs="EPSG:4326")

def _meters_per_degree(lat_deg: float) -> tuple[float, float]:
    lat_rad = math.radians(lat_deg)
    mx = 111_320.0 * math.cos(lat_rad)
    my = 110_540.0
    return mx, my

def _sample_dem_and_slope(dem_path: Path, gdf: gpd.GeoDataFrame) -> tuple[list[float], list[float]]:
    elevs, slopes = [], []
    try:
        with rasterio.open(dem_path) as src:
            tr: Affine = src.transform
            band1 = src.read(1, masked=True)
            deg_based = abs(tr.a) < 1e-2 and abs(tr.e) < 1e-2
            for pt in gdf.geometry:
                if pt is None or pt.is_empty:
                    elevs.append(np.nan); slopes.append(np.nan); continue
                col, row = src.index(pt.x, pt.y)
                if 0 <= row < src.height and 0 <= col < src.width:
                    z = band1[row, col]
                    z = float(z) if z is not np.ma.masked else np.nan
                else:
                    z = np.nan
                r0, r1 = max(row-1, 0), min(row+2, src.height)
                c0, c1 = max(col-1, 0), min(col+2, src.width)
                patch = band1[r0:r1, c0:c1]
                slope_deg = np.nan
                if patch.shape == (3,3) and np.all(~patch.mask):
                    if deg_based:
                        mx, my = _meters_per_degree(pt.y)
                        px = abs(tr.a) * mx; py = abs(tr.e) * my
                    else:
                        px = abs(tr.a); py = abs(tr.e) if abs(tr.e) > 0 else abs(tr.d)
                    p = patch.filled(np.nan).astype(float)
                    dzdx = ((p[0,2] + 2*p[1,2] + p[2,2]) - (p[0,0] + 2*p[1,0] + p[2,0])) / (8.0 * px)
                    dzdy = ((p[2,0] + 2*p[2,1] + p[2,2]) - (p[0,0] + 2*p[0,1] + p[0,2])) / (8.0 * py)
                    slope_rad = math.atan(math.hypot(dzdx, dzdy))
                    slope_deg = math.degrees(slope_rad)
                elevs.append(z); slopes.append(slope_deg)
        log.info("dem_sampled", count=len(elevs))
    except Exception as e:
        log.warning("dem_sample_failed", err=str(e))
        elevs = [np.nan] * len(gdf); slopes = [np.nan] * len(gdf)
    return elevs, slopes

def _sample_categorical(raster_path: Path, gdf: gpd.GeoDataFrame) -> list[float]:
    vals: list[float] = []
    try:
        with rasterio.open(raster_path) as src:
            for pt in gdf.geometry:
                if pt is None or pt.is_empty:
                    vals.append(np.nan); continue
                val = list(src.sample([(pt.x, pt.y)]))[0][0]
                if src.nodata is not None and val == src.nodata:
                    vals.append(np.nan)
                else:
                    try: vals.append(int(val))
                    except Exception: vals.append(float(val))
        log.info("landcover_sampled", count=len(vals))
    except Exception as e:
        log.warning("landcover_sample_failed", err=str(e))
        vals = [np.nan] * len(gdf)
    return vals

# --- enrich fires with elevation, slope, land cover ---
try:
    elev, slope = _sample_dem_and_slope(DEM_TIF, gdf_fires)
    gdf_fires["elevation_m"]  = elev
    gdf_fires["slope_deg"]    = slope
    gdf_fires["land_cover_code"] = _sample_categorical(LC_TIF, gdf_fires)

    gdf_fires.drop(columns=["geometry"]).to_parquet(fires_terrain_out, index=False)
    log.info("fires_terrain_saved", path=str(fires_terrain_out), rows=len(gdf_fires))
    print(f"Terrain-enriched fires saved: {fires_terrain_out}")
except Exception as e:
    log.error("fires_terrain_failed", err=str(e))
    raise


2025-09-30 20:22:56 [info     ] dem_present                    path=/Users/evareysanchez/WildfiresAI/data/terrain/copernicus_dem.tif region=ES size=136749593
2025-09-30 20:22:57 [info     ] worldcover_found               count=23 region=ES year=2021
2025-09-30 20:22:59 [info     ] landcover_clipped              path=/Users/evareysanchez/WildfiresAI/data/terrain/esa_worldcover_clip.tif region=ES shape=(22800, 6000)
2025-09-30 20:23:00 [warning  ] dem_sample_failed              err='Cannot convert fill_value nan to dtype int16' region=ES
2025-09-30 20:23:00 [info     ] landcover_sampled              count=737 region=ES
2025-09-30 20:23:00 [info     ] fires_terrain_saved            path=/Users/evareysanchez/WildfiresAI/data/processed/fires_terrain_ES_2025-09-23_2025-09-30.parquet region=ES rows=737
Terrain-enriched fires saved: /Users/evareysanchez/WildfiresAI/data/processed/fires_terrain_ES_2025-09-23_2025-09-30.parquet


## Cell 6 — Test OpenAI Connection (Optional, Paid)

This cell performs a **sanity check** of the OpenAI client connection.  
- It sends a lightweight test prompt to the `gpt-4o-mini` model.  
- The model is asked to produce a short haiku on *AI and wildfire resilience*.  
- This verifies that the authentication and request pipeline are working.  

**Note**: Executing this cell will consume API credits.  
It should be run only once to validate connectivity and left commented or skipped in regular workflows.


In [ ]:
# Optional test (may consume credits). 
resp = client.responses.create(
    model="gpt-4o-mini", 
    input="Write a 3-line haiku about AI and wildfire resilience."
)
print("Model response:\n")
print(resp.output_text)


## Cell 7 — Verify FIRMS Token (NASA FIRMS)

This cell verifies that the **NASA FIRMS Bearer token** is available in the environment.  
- The token is required for accessing FIRMS API endpoints (satellite fire detection data).  
- It is loaded from the shell configuration file (`~/.zshrc`) into the environment.  
- For security, only the first characters of the token are displayed.  

If the token is missing, the user must check the shell configuration and reload the session with:

```bash
source ~/.zshrc


In [ ]:
from pathlib import Path
import os, math
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.transform import Affine
import structlog

log = structlog.get_logger("geoterrain").bind(region=os.getenv("WF_REGION", "ES"))

# --- config / paths
REGION = os.getenv("WF_REGION", "ES")
W, S, E, N = globals().get("ACTIVE_BBOX", globals().get("SPAIN_BBOX", (-9.5, 35.0, 3.5, 43.9)))

TERRAIN_DIR = PROJECT_ROOT / "data" / "terrain"
TERRAIN_DIR.mkdir(parents=True, exist_ok=True)
DEM_TIF = TERRAIN_DIR / "copernicus_dem.tif"
LC_TIF  = TERRAIN_DIR / "esa_worldcover_clip.tif"  # clipped to bbox

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
fires_clean_path   = PROCESSED_DIR / f"firms_clean_{REGION}_{DATE_FROM}_{DATE_TO}.csv"
fires_terrain_out  = PROCESSED_DIR / f"fires_terrain_{REGION}_{DATE_FROM}_{DATE_TO}.parquet"

# ----------------------- helpers: DEM download -----------------------

def _download_dem_opentopo(w: float, s: float, e: float, n: float, out_path: Path, demtype="SRTMGL3") -> None:
    """Fetch DEM GeoTIFF clipped to bbox using OpenTopography Global DEM API (requires OPENTOPO_API_KEY)."""
    import requests
    api_key = os.getenv("OPENTOPO_API_KEY")
    if not api_key:
        raise RuntimeError("Missing OPENTOPO_API_KEY in .env")
    url = "https://portal.opentopography.org/API/globaldem"
    params = {
        "demtype": demtype, "south": s, "north": n, "west": w, "east": e,
        "outputFormat": "GTiff", "API_Key": api_key
    }
    r = requests.get(url, params=params, timeout=600); r.raise_for_status()
    try:
        js = r.json()
        tif_url = None
        if isinstance(js, dict) and "result" in js and "links" in js["result"]:
            for lnk in js["result"]["links"]:
                href = lnk.get("href", "")
                if href.endswith(".tif") or lnk.get("rel") in ("data", "download", "self"):
                    tif_url = href; break
        if not tif_url:
            raise RuntimeError(f"No GeoTIFF link in OpenTopography response: {js}")
        r2 = requests.get(tif_url, timeout=1200); r2.raise_for_status()
        out_path.write_bytes(r2.content)
        log.info("dem_downloaded", src=tif_url, path=str(out_path))
    except Exception:
        # Direct binary fallback (some endpoints return TIFF directly)
        if r.headers.get("content-type","").lower().startswith("image/tiff"):
            out_path.write_bytes(r.content)
            log.info("dem_downloaded_direct", bytes=len(r.content), path=str(out_path))
        else:
            raise

# ----------------------- helpers: WorldCover build -----------------------

def _build_worldcover_from_stac(w: float, s: float, e: float, n: float, out_path: Path) -> None:
    """
    Query ESA WorldCover via Planetary Computer STAC, mosaic intersecting tiles and clip to bbox.
    Writes a single GeoTIFF to `out_path`. Requires `pystac-client` and `planetary_computer`.
    """
    from pystac_client import Client
    import planetary_computer as pc
    import rasterio
    from rasterio.merge import merge as rio_merge
    import numpy as np

    stac = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    bbox = [w, s, e, n]

    # Years available in WorldCover (as of v200: 2020, 2021)
    years = [2021, 2020]
    items = []
    last_err = None

    for yr in years:
        try:
            search = stac.search(
                collections=["esa-worldcover"],
                bbox=bbox,
                datetime=f"{yr}-01-01/{yr}-12-31"
            )
            items = list(search.items())
            if items:
                log.info("worldcover_found", year=yr, count=len(items))
                break
        except Exception as ex:
            last_err = ex
            continue

    if not items:
        raise RuntimeError(f"No WorldCover found for bbox in years={years}. Last error: {last_err}")

    datasets = []
    try:
        for it in items:
            asset = it.assets.get("map") or it.assets.get("Map")
            if not asset:
                continue
            signed_href = pc.sign(asset.href)
            datasets.append(rasterio.open(signed_href))
        if not datasets:
            raise RuntimeError("WorldCover items found but no usable 'map' assets.")

        mosaic, transform = rio_merge(datasets)
        profile = datasets[0].profile.copy()
        profile.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": transform,
            "count": 1,
            "dtype": mosaic.dtype,
            "compress": "lzw"
        })

        out_path.parent.mkdir(parents=True, exist_ok=True)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(mosaic[0], 1)  # categorical band
        log.info("landcover_built", tiles=len(datasets), path=str(out_path))
    finally:
        for ds in datasets:
            try:
                ds.close()
            except Exception:
                pass


# ----------------------- ensure assets -----------------------

try:
    if not DEM_TIF.exists() or DEM_TIF.stat().st_size < 100_000:
        _download_dem_opentopo(W, S, E, N, DEM_TIF)
    else:
        log.info("dem_present", path=str(DEM_TIF), size=DEM_TIF.stat().st_size)
except Exception as e:
    log.error("dem_unavailable", err=str(e))

try:
    if not LC_TIF.exists() or LC_TIF.stat().st_size < 100_000:
        _build_worldcover_from_stac(W, S, E, N, LC_TIF)
    else:
        log.info("landcover_present", path=str(LC_TIF), size=LC_TIF.stat().st_size)
except Exception as e:
    log.error("landcover_unavailable", err=str(e))

# ----------------------- load fires & sampling -----------------------

try:
    df_fires = pd.read_csv(fires_clean_path)
    gdf_fires = gpd.GeoDataFrame(df_fires,
                                 geometry=gpd.points_from_xy(df_fires.lon, df_fires.lat),
                                 crs="EPSG:4326")
except Exception as e:
    log.error("fires_load_failed", err=str(e))
    gdf_fires = gpd.GeoDataFrame(columns=["lat","lon","geometry"], crs="EPSG:4326")

def _meters_per_degree(lat_deg: float) -> tuple[float,float]:
    lat_rad = math.radians(lat_deg)
    mx = 111_320.0 * math.cos(lat_rad)
    my = 110_540.0
    return mx, my

def _sample_dem_and_slope(dem_path: Path, gdf: gpd.GeoDataFrame) -> tuple[list[float], list[float]]:
    """Sample elevation and slope (Horn 3×3) for each fire point."""
    elevs, slopes = [], []
    try:
        with rasterio.open(dem_path) as src:
            tr: Affine = src.transform
            deg_based = abs(tr.a) < 1e-2 and abs(tr.e) < 1e-2
            band1 = src.read(1, masked=True)
            for pt in gdf.geometry:
                if pt is None or pt.is_empty:
                    elevs.append(np.nan); slopes.append(np.nan); continue
                col, row = src.index(pt.x, pt.y)
                if 0 <= row < src.height and 0 <= col < src.width:
                    z = band1[row, col]
                    z = float(z) if z is not np.ma.masked else np.nan
                else:
                    z = np.nan
                r0, r1 = max(row-1,0), min(row+2,src.height)
                c0, c1 = max(col-1,0), min(col+2,src.width)
                patch = band1[r0:r1, c0:c1]
                slope_deg = np.nan
                if patch.shape==(3,3) and np.all(~patch.mask):
                    if deg_based:
                        mx, my = _meters_per_degree(pt.y)
                        px, py = abs(tr.a)*mx, abs(tr.e)*my
                    else:
                        px, py = abs(tr.a), (abs(tr.e) if abs(tr.e)>0 else abs(tr.d))
                    p = patch.filled(np.nan).astype(float)
                    dzdx = ((p[0,2]+2*p[1,2]+p[2,2])-(p[0,0]+2*p[1,0]+p[2,0]))/(8.0*px)
                    dzdy = ((p[2,0]+2*p[2,1]+p[2,2])-(p[0,0]+2*p[0,1]+p[0,2]))/(8.0*py)
                    slope_deg = math.degrees(math.atan(math.hypot(dzdx,dzdy)))
                elevs.append(z); slopes.append(slope_deg)
        log.info("dem_sampled", count=len(elevs))
    except Exception as e:
        log.warning("dem_sample_failed", err=str(e))
        elevs=[np.nan]*len(gdf); slopes=[np.nan]*len(gdf)
    return elevs, slopes

def _sample_categorical(raster_path: Path, gdf: gpd.GeoDataFrame) -> list[float]:
    """Nearest-neighbor sampling for categorical rasters (e.g., WorldCover)."""
    vals=[]
    try:
        with rasterio.open(raster_path) as src:
            nodata=src.nodata
            for pt in gdf.geometry:
                if pt is None or pt.is_empty:
                    vals.append(np.nan); continue
                val = list(src.sample([(pt.x,pt.y)]))[0][0]
                if nodata is not None and val==nodata:
                    vals.append(np.nan)
                else:
                    try: vals.append(int(val))
                    except: vals.append(float(val))
        log.info("landcover_sampled", count=len(vals))
    except Exception as e:
        log.warning("landcover_sample_failed", err=str(e))
        vals=[np.nan]*len(gdf)
    return vals

# --- enrich and save
try:
    elev, slope = _sample_dem_and_slope(DEM_TIF, gdf_fires)
    gdf_fires["elevation_m"] = elev
    gdf_fires["slope_deg"] = slope
    gdf_fires["land_cover_code"] = _sample_categorical(LC_TIF, gdf_fires)
    gdf_fires.drop(columns=["geometry"]).to_parquet(fires_terrain_out, index=False)
    log.info("fires_terrain_saved", path=str(fires_terrain_out), rows=len(gdf_fires))
    print(f"Terrain-enriched fires → {fires_terrain_out}")
except Exception as e:
    log.error("fires_terrain_failed", err=str(e))
    raise

2025-09-30 20:13:49 [info     ] dem_present                    path=/Users/evareysanchez/WildfiresAI/data/terrain/copernicus_dem.tif region=ES size=136749593
2025-09-30 20:13:50 [info     ] worldcover_found               count=23 region=ES year=2021


## Cell 8 — Verify FIRMS MAP_KEY (NASA FIRMS)

This cell verifies that the **NASA FIRMS MAP_KEY** (legacy API key) is available in the environment.  
- The MAP_KEY is required for the **CSV-style FIRMS API**, where the key is embedded directly in the request URL.  
- It is stored in the shell configuration (`~/.zshrc`) and loaded into the environment for use in this notebook.  
- For security, only the first characters of the key are displayed.  

If the MAP_KEY is not found, the pipeline raises an explicit error instructing the user to update and reload their environment:

```bash
export FIRMS_MAP_KEY="your_key_here"
source ~/.zshrc


## Cell 9 — Connect to FIRMS API (MAP_KEY mode)

This cell establishes a connection to the **NASA FIRMS API** using the legacy **MAP_KEY** authentication scheme.  
Key operations performed here:

1. **Authentication**  
   - The `FIRMS_MAP_KEY` is embedded directly in the API request URL.  
   - Ensures access to FIRMS CSV endpoints even if Bearer token support is unavailable.

2. **Spatial & Temporal Filtering**  
   - The request is bounded by the Spain region (defined in `SPAIN_BBOX`).  
   - The temporal window is restricted to the **last 7 days**, consistent with FIRMS near-real-time (NRT) availability.  

3. **Data Retrieval**  
   - Fire detections are retrieved from the **VIIRS SNPP NRT** product.  
   - The response is parsed into a Pandas DataFrame and validated.  

4. **Persistence & Preview**  
   - The raw CSV is saved to the `data/raw/` directory with a timestamped filename.  
   - A quick preview (`head(5)`) is displayed to confirm structure and content.

This step ensures the pipeline begins with authoritative, high-resolution fire detection data, suitable for downstream cleaning, integration with weather datasets, and scientific validation.




In [ ]:
from io import StringIO  # only needed here

# Ensure MAP_KEY is available (from Cell 7)
if not FIRMS_MAP_KEY:
    raise RuntimeError("FIRMS_MAP_KEY not found. Did you export it in ~/.zshrc?")

# Spain bounding box (W, S, E, N)
w, s, e, n = SPAIN_BBOX

# FIRMS product and temporal window
product = "VIIRS_SNPP_NRT"
days = 7  # valid range: 1..10

out_csv = RAW_DIR / "firms_last7d_es_raw.csv"

# MAP_KEY style (token in path)
url = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{FIRMS_MAP_KEY}/{product}/world/{days}"
params = {"west": w, "south": s, "east": e, "north": n}

r = requests.get(url, params=params, timeout=60)
if r.status_code != 200 or "Invalid" in r.text[:200]:
    raise RuntimeError(f"FIRMS API request failed: HTTP {r.status_code} · {r.text[:200]}")

# Parse and save
df_raw = pd.read_csv(StringIO(r.text), dtype={"acq_time": "string"})
df_raw.to_csv(out_csv, index=False)

print("FIRMS API connection successful (MAP_KEY mode)")
print(f"Rows fetched: {len(df_raw)}")
print(f"Saved raw CSV → {out_csv}")

display(df_raw.head(5))



## Cell 10 — Open-Meteo Helper (Hourly, ERA5 Archive)

This cell defines a reusable **helper function** to query the [Open-Meteo Archive API](https://open-meteo.com/en/docs/historical-weather-api) for historical weather data based on the **ERA5 reanalysis dataset**. No API key is required.  

**Main features:**

1. **Endpoint & Variables**  
   - Uses the ERA5 archive endpoint (`https://archive-api.open-meteo.com/v1/archive`).  
   - Retrieves hourly variables relevant to wildfire dynamics (temperature, humidity, wind, precipitation, and surface pressure).  

2. **Function `fetch_open_meteo`**  
   - Inputs: latitude, longitude, start date, end date, and selected variables.  
   - Returns a **Pandas DataFrame** with hourly weather records in UTC.  
   - Columns include `time_utc`, meteorological variables, and coordinates (`lat`, `lon`).  

3. **Error Handling & Validation**  
   - Raises explicit errors if the API response is invalid (e.g., missing hourly data).  
   - Ensures numeric casting of meteorological fields for consistent analysis.  

4. **Persistence**  
   - Provides a lightweight `save_df` utility to persist datasets under the `data/raw/` folder.  
   - Facilitates traceability and reproducibility of weather datasets.  

This component is critical for aligning **FIRMS fire detections** with **meteorological context**, enabling joint analyses (e.g., fire spread vs. weather dynamics) and subsequent model training.



In [ ]:
from typing import Iterable

# Endpoint correcto para histórico/reanálisis (ERA5):
OPEN_METEO_BASE = "https://archive-api.open-meteo.com/v1/archive"

# Variables horarias por defecto (puedes añadir/quitar)
OPEN_METEO_HOURLY = [
    "temperature_2m",
    "relative_humidity_2m",
    "dew_point_2m",
    "windspeed_10m",
    "windgusts_10m",
    "winddirection_10m",
    "precipitation",
    "surface_pressure",
]

# Pequeño helper local por si save_df aún no existe
if "save_df" not in globals():
    import pandas as pd
    from pathlib import Path
    def save_df(df: pd.DataFrame, path: Path) -> None:
        path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(path, index=False)
        print(f"Saved: {path}")

def fetch_open_meteo(
    lat: float,
    lon: float,
    date_from: date,
    date_to: date,
    hourly: Iterable[str] = OPEN_METEO_HOURLY,
    model: str | None = "era5",  # "era5" o "era5_land"; None para auto
) -> pd.DataFrame:
    """Fetch hourly weather (UTC) for a point and date window using ERA5 archive."""
    params = {
        "latitude":  round(float(lat), 5),
        "longitude": round(float(lon), 5),
        "start_date": str(date_from),
        "end_date":   str(date_to),
        "hourly":     ",".join(hourly),
        "timezone":   "UTC",
    }
    if model:
        params["models"] = model

    r = requests.get(OPEN_METEO_BASE, params=params, timeout=60)
    if r.status_code != 200:
        raise RuntimeError(f"Open-Meteo error {r.status_code}: {r.text[:200]}")

    js = r.json()
    if "hourly" not in js or "time" not in js["hourly"]:
        raise RuntimeError("Open-Meteo response missing hourly data.")

    df = pd.DataFrame(js["hourly"]).rename(columns={"time": "time_utc"})
    df["time_utc"] = pd.to_datetime(df["time_utc"], utc=True, errors="coerce")
    df["lat"] = round(float(lat), 5)
    df["lon"] = round(float(lon), 5)

    # Asegurar tipos numéricos
    for c in df.columns:
        if c not in ("time_utc", "lat", "lon"):
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df



## Cell 11 — Open-Meteo Smoke Test (Spain Centroid)

This cell performs a **sanity check** of the Open-Meteo integration by querying historical weather data at the **centroid of Spain’s bounding box**. The goal is to validate that the pipeline can fetch, persist, and visualize meteorological information.  

**Main steps:**

1. **Centroid Calculation**  
   - Latitude and longitude are derived from the midpoint of Spain’s bounding box (`SPAIN_BBOX`, defined in Cell 3).  

2. **Weather Fetching**  
   - Calls `fetch_open_meteo` (Cell 10) to retrieve ERA5 reanalysis data between `DATE_FROM` and `DATE_TO`.  
   - Produces an **hourly-resolution DataFrame** with variables such as temperature, humidity, wind, and precipitation.  

3. **Persistence**  
   - Saves raw hourly data under `data/raw/`.  
   - Computes a **daily aggregated summary** (mean temperature, mean humidity, mean windspeed, total precipitation) and stores it under `data/processed/`.  

4. **Visualization**  
   - Generates a **time-series plot of temperature** at the centroid (°C vs UTC time).  
   - Saves the figure under the `reports/` directory for reproducibility and reporting.  

This smoke test confirms that **Open-Meteo is properly integrated**, and it sets the foundation for linking weather conditions with FIRMS fire detections in later steps.



In [ ]:
# Centroid of Spain bbox (W, S, E, N) from Global Configuration (Cell 3)
W, S, E, N = SPAIN_BBOX
lat_c = (S + N) / 2.0
lon_c = (W + E) / 2.0

# Fetch hourly weather (UTC)
df_weather_centroid = fetch_open_meteo(lat_c, lon_c, DATE_FROM, DATE_TO, model="era5")

# Save raw weather CSV under data/raw/
raw_weather_path = RAW_DIR / f"openmeteo_es_centroid_{DATE_FROM}_{DATE_TO}.csv"
save_df(df_weather_centroid, raw_weather_path)

# Display quick preview
print(f"Rows (hourly): {len(df_weather_centroid)}")
display(df_weather_centroid.head(10))

# Also save a daily summary (mean temperature, etc.) to processed/
daily_summary = (
    df_weather_centroid.assign(day=df_weather_centroid["time_utc"].dt.floor("D"))
    .groupby("day")
    .agg(
        temp_mean=("temperature_2m", "mean"),
        rh_mean=("relative_humidity_2m", "mean"),
        wind_mean=("windspeed_10m", "mean"),
        precip_sum=("precipitation", "sum"),
    )
    .reset_index()
)
daily_out = PROCESSED_DIR / f"openmeteo_es_centroid_daily_{DATE_FROM}_{DATE_TO}.csv"
save_df(daily_summary, daily_out)

# Plot: Temperature vs time (and save figure)
plt.figure(figsize=(9,4))
plt.plot(df_weather_centroid["time_utc"], df_weather_centroid["temperature_2m"])
plt.title("Open-Meteo — Temperature at Spain centroid (UTC)")
plt.xlabel("Time (UTC)")
plt.ylabel("Temperature 2m (°C)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Save figure to reports/
fig_path = REPORTS_DIR / f"openmeteo_temp_centroid_{DATE_FROM}_{DATE_TO}.png"
plt.figure(figsize=(9,4))
plt.plot(df_weather_centroid["time_utc"], df_weather_centroid["temperature_2m"])
plt.title("Open-Meteo — Temperature at Spain centroid (UTC)")
plt.xlabel("Time (UTC)")
plt.ylabel("Temperature 2m (°C)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(fig_path, dpi=160)
plt.close()
print(f"Saved figure: {fig_path}")



## Cell 12 — Open-Meteo at First Fire Location (Optional)

This cell enriches the FIRMS detections by fetching weather conditions at the **geographic location of the first fire point** available. It is optional, but critical for validating the integration of fire and meteorological data.

**Main steps:**

1. **Candidate Fire Location**  
   - Attempts to extract latitude/longitude from `df_raw` (raw FIRMS) or `df_fires` (processed FIRMS).  
   - Accepts flexible column names (`latitude/longitude`, `lat/lon`, `x/y`).  

2. **Weather Retrieval**  
   - If a valid fire location exists, queries Open-Meteo’s ERA5 archive for hourly conditions.  
   - Produces a DataFrame (`df_weather_point`) with temperature, humidity, windspeed, precipitation, and more.  

3. **Persistence**  
   - Saves raw hourly weather data under `data/raw/`.  
   - Aggregates to **daily summaries** (mean temperature, mean humidity, mean windspeed, precipitation sum) and saves under `data/processed/`.  

4. **Visualization**  
   - Generates and displays a **time-series chart** of temperature at the fire location.  
   - Saves the chart under `reports/` for later reporting and analysis.  

This optional step serves as a **prototype for future joins**: linking individual fire detections with their local weather context. It provides a first look at how meteorology and active fire events intersect in the pipeline.


In [ ]:
candidate_lat, candidate_lon = None, None

if "df_raw" in globals() and isinstance(df_raw, pd.DataFrame) and not df_raw.empty:
    cand = df_raw.rename(columns=str.lower)
    for a, b in (("latitude","longitude"), ("lat","lon"), ("y","x")):
        if a in cand.columns and b in cand.columns:
            candidate_lat = float(cand.iloc[0][a])
            candidate_lon = float(cand.iloc[0][b])
            break

elif "df_fires" in globals() and isinstance(df_fires, pd.DataFrame) and not df_fires.empty:
    candidate_lat = float(df_fires.iloc[0]["latitude"])
    candidate_lon = float(df_fires.iloc[0]["longitude"])

if candidate_lat is not None and candidate_lon is not None:
    df_weather_point = fetch_open_meteo(candidate_lat, candidate_lon, DATE_FROM, DATE_TO, model="era5")

    # Save raw weather CSV
    raw_pt_path = RAW_DIR / f"openmeteo_at_fire_{DATE_FROM}_{DATE_TO}.csv"
    save_df(df_weather_point, raw_pt_path)

    # Daily summary → processed
    daily_summary_fire = (
        df_weather_point.assign(day=df_weather_point["time_utc"].dt.floor("D"))
        .groupby("day")
        .agg(
            temp_mean=("temperature_2m", "mean"),
            rh_mean=("relative_humidity_2m", "mean"),
            wind_mean=("windspeed_10m", "mean"),
            precip_sum=("precipitation", "sum"),
        )
        .reset_index()
    )
    daily_pt_out = PROCESSED_DIR / f"openmeteo_at_fire_daily_{DATE_FROM}_{DATE_TO}.csv"
    save_df(daily_summary_fire, daily_pt_out)

    print(f"Weather fetched at fire point lat={candidate_lat:.4f}, lon={candidate_lon:.4f}")
    preview(df_weather_point)

    # Plot: Temperature vs time
    plt.figure(figsize=(9,4))
    plt.plot(df_weather_point["time_utc"], df_weather_point["temperature_2m"])
    plt.title(f"Open-Meteo — Temperature at fire point ({candidate_lat:.2f},{candidate_lon:.2f})")
    plt.xlabel("Time (UTC)")
    plt.ylabel("Temperature 2m (°C)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    # Save figure
    fig_path = REPORTS_DIR / f"openmeteo_firepoint_temp_{DATE_FROM}_{DATE_TO}.png"
    plt.figure(figsize=(9,4))
    plt.plot(df_weather_point["time_utc"], df_weather_point["temperature_2m"])
    plt.title(f"Open-Meteo — Temperature at fire point ({candidate_lat:.2f},{candidate_lon:.2f})")
    plt.xlabel("Time (UTC)")
    plt.ylabel("Temperature 2m (°C)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()




## Cell 13 — Build Clean FIRMS DataFrame (df_fires)

Purpose. Convert the raw FIRMS CSV (Cell 9) into a clean, analysis-ready table:

normalize schema and types

construct a UTC timestamp (acq_datetime)

filter by Spain bbox and DATE_FROM → DATE_TO (UTC)

drop duplicates and sort

persist the result to data/processed/ and expose it as df_fires

In [ ]:
# --- Guards -------------------------------------------------------------------
import pandas as pd
import numpy as np

assert "SPAIN_BBOX" in globals(), "Missing SPAIN_BBOX (run Global Configuration)."
assert "DATE_FROM" in globals() and "DATE_TO" in globals(), "Missing DATE_FROM/DATE_TO."
assert "PROCESSED_DIR" in globals(), "Missing PROCESSED_DIR (run Global Configuration)."

# Prefer df_raw from Cell 9; otherwise try to read the saved raw CSV
if "df_raw" not in globals() or not isinstance(df_raw, pd.DataFrame) or df_raw.empty:
    from pathlib import Path
    raw_fallback = (RAW_DIR / "firms_last7d_es_raw.csv")
    if raw_fallback.exists():
        df_raw = pd.read_csv(raw_fallback, dtype={"acq_time": "string"})
        print(f"Loaded raw fallback: {raw_fallback}")
    else:
        raise RuntimeError("No FIRMS raw dataframe found. Run Cell 9 first.")

# --- Helpers ------------------------------------------------------------------
def _parse_acq_datetime(acq_date, acq_time) -> pd.Series:
    """Combine acq_date (YYYY-MM-DD) and acq_time (HHMM) into a UTC timestamp."""
    t = pd.Series(acq_time, dtype="string").str.zfill(4)
    dt = pd.Series(acq_date, dtype="string") + " " + t.str[:2] + ":" + t.str[2:] + ":00"
    return pd.to_datetime(dt, utc=True, errors="coerce")

def _clean_firms(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    df.columns = [c.lower() for c in df.columns]

    # flexible renames (handles common variants)
    rename_map = {
        "lat": "latitude", "long": "longitude", "lon": "longitude",
        "brightness": "brightness", "bright_ti4": "bright_ti4", "bright_ti5": "bright_ti5",
        "acq_date": "acq_date", "acq_time": "acq_time",
        "confidence": "confidence", "daynight": "daynight",
        "satellite": "satellite", "instrument": "instrument",
        "scan": "scan", "track": "track", "version": "version", "frp": "frp"
    }
    df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)

    # required columns
    req = ["latitude", "longitude", "acq_date", "acq_time"]
    missing = [c for c in req if c not in df.columns]
    if missing:
        raise ValueError(f"Raw FIRMS is missing required columns: {missing}")

    # timestamp
    df["acq_datetime"] = _parse_acq_datetime(df["acq_date"], df["acq_time"])

    # numeric coercion
    for c in ["latitude","longitude","frp","bright_ti4","bright_ti5","brightness","scan","track"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # confidence normalization (text → numeric if needed)
    if "confidence" in df.columns:
        if df["confidence"].dtype == "object" or str(df["confidence"].dtype).startswith("string"):
            map_text = {"low": 20, "nominal": 50, "high": 85}
            df["confidence_text"] = df["confidence"].str.lower().map(lambda x: x if x in map_text else np.nan)
            df["confidence_num"]  = df["confidence"].str.lower().map(map_text)
        else:
            df["confidence_num"] = pd.to_numeric(df["confidence"], errors="coerce")
    else:
        df["confidence_num"] = np.nan

    return df

def _clip_bbox(df: pd.DataFrame, bbox) -> pd.DataFrame:
    w, s, e, n = bbox
    m = df["longitude"].between(w, e) & df["latitude"].between(s, n)
    return df.loc[m].copy()

def _clip_timerange(df: pd.DataFrame, start_utc: pd.Timestamp, end_utc_exclusive: pd.Timestamp) -> pd.DataFrame:
    m = (df["acq_datetime"] >= start_utc) & (df["acq_datetime"] < end_utc_exclusive)
    return df.loc[m].copy()

# --- Clean + Filter -----------------------------------------------------------
df_tmp = _clean_firms(df_raw)

# time window as UTC [start, end)
start_utc = pd.Timestamp(DATE_FROM).tz_localize("UTC")
end_utc_exclusive = pd.Timestamp(DATE_TO + pd.Timedelta(days=1)).tz_localize("UTC")

df_tmp = _clip_bbox(df_tmp, SPAIN_BBOX)
df_tmp = _clip_timerange(df_tmp, start_utc, end_utc_exclusive)

# de-duplicate & sort
df_fires = (
    df_tmp.drop_duplicates(subset=["acq_datetime", "latitude", "longitude"])
          .sort_values("acq_datetime")
          .reset_index(drop=True)
)

# --- Persist & Report ---------------------------------------------------------
out_path = PROCESSED_DIR / f"firms_es_{DATE_FROM}_{DATE_TO}.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
df_fires.to_csv(out_path, index=False)

print(" FIRMS cleaned dataframe ready → df_fires")
print(f"Rows: {len(df_fires)}  |  Saved: {out_path}")
if len(df_fires):
    print(f"Time range (UTC): {df_fires['acq_datetime'].min()} → {df_fires['acq_datetime'].max()}")
    print(f"BBox: {SPAIN_BBOX}")

# Preview
display(df_fires.head(min(10, len(df_fires))))


## Cell 14 — Data Inventory (with last modified + CSV)

This cell performs a **systematic inventory of the project’s data folders** to ensure full traceability of the pipeline. It inspects the contents of `data/raw/` and `data/processed/`, reports file sizes and modification timestamps, and consolidates the results into a structured DataFrame.

**Main steps:**

1. **Directory Resolution**  
   - Supports multiple naming conventions (`RAW_DIR` / `DATA_RAW`, `PROCESSED_DIR` / `DATA_PROC`).  
   - Ensures project directories are initialized before proceeding.  

2. **Inventory Generation**  
   - Iterates through all `.csv` files in the raw and processed folders.  
   - Extracts **file name, size (KB), and last modified time**.  
   - Appends results to a unified list for reporting.  

3. **Persistence**  
   - Saves the inventory itself as a CSV file under `data/processed/` → `data_inventory.csv`.  
   - This creates a **meta-trace** of all generated artifacts, useful for reproducibility and audits.  

4. **Visualization**  
   - Prints a human-readable summary to the console.  
   - Displays the first 20 rows of the inventory DataFrame inside the notebook.  

This step ensures that the pipeline maintains a **transparent record of all inputs and outputs** generated so far, a requirement for scientific reproducibility and future integration with automated QA systems.


In [ ]:
import pandas as pd
from pathlib import Path

# 1) Ensure raw FIRMS dataframe is available; if not, reload from the saved CSV (Cell 9 output)
raw_path = RAW_DIR / "firms_last7d_es_raw.csv"
if "df_raw" not in globals() or df_raw is None or df_raw.empty:
    if raw_path.exists():
        df_raw = pd.read_csv(raw_path, dtype={"acq_time": "string"})
        print(f"Loaded raw FIRMS from {raw_path}")
    else:
        raise RuntimeError("df_raw is missing and raw CSV not found. Run Cell 9 first.")

# 2) Normalize column names to lowercase
df = df_raw.rename(columns=str.lower).copy()

# 3) Basic schema validation: ensure required fields are present
required = {"latitude", "longitude", "acq_date", "acq_time"}
missing = [c for c in required if c not in df.columns]
if missing:
    raise RuntimeError(f"Raw FIRMS missing columns: {missing}")

# 4) Build UTC acquisition datetime from acq_date + acq_time
t = pd.Series(df["acq_time"], dtype="string").str.zfill(4)
dt = pd.Series(df["acq_date"], dtype="string") + " " + t.str[:2] + ":" + t.str[2:] + ":00"
df["acq_datetime"] = pd.to_datetime(dt, utc=True, errors="coerce")

# 5) Convert key numeric columns to proper dtypes
for c in ("latitude", "longitude", "frp", "bright_ti4", "bright_ti5", "scan", "track"):
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 6) Apply spatial (Spain bounding box) and temporal filters
W, S, E, N = SPAIN_BBOX
m_geo = df["longitude"].between(W, E) & df["latitude"].between(S, N)
m_time = (df["acq_datetime"] >= pd.to_datetime(DATE_FROM).tz_localize("UTC")) & \
         (df["acq_datetime"] <= pd.to_datetime(DATE_TO).tz_localize("UTC"))

df_fires = df.loc[m_geo & m_time].copy()

# 7) Drop duplicates and sort chronologically
df_fires = (
    df_fires.drop_duplicates(subset=["acq_datetime", "latitude", "longitude"])
            .sort_values("acq_datetime")
            .reset_index(drop=True)
)

print(f" df_fires built. Rows: {len(df_fires)}")

# 8) Save cleaned dataset to processed folder
out_file = PROCESSED_DIR / f"fires_firms_es_{DATE_FROM}_{DATE_TO}.csv"
df_fires.to_csv(out_file, index=False)
print("Saved:", out_file)

# 9) Quick preview of the cleaned dataset
display(df_fires.head(10))


## Cell 15 — Fetch Open-Meteo for Fire Points (robust, cached)

This cell enriches **FIRMS fire detections with localized weather conditions** by systematically querying the Open-Meteo ERA5 archive for each unique fire point. Compared to a naive batch approach, this implementation is designed for **resilience and reproducibility**.

**Main steps:**

1. **Preconditions**  
   - Verifies that `df_fires` exists and is non-empty.  
   - Ensures the pipeline only queries weather data when fire detections are available.  

2. **Unique Fire Locations**  
   - Extracts latitude/longitude pairs from FIRMS detections.  
   - Rounds coordinates to 0.01° (~1.1 km) to avoid redundant requests while preserving spatial fidelity.  
   - Drops duplicates, keeping only distinct fire points.  

3. **Sampling Strategy & Limits**  
   - Caps the number of processed points (`MAX_POINTS = 15` by default).  
   - Prevents **API throttling** and keeps runtime manageable.  
   - Prints the total vs. processed points for transparency.  

4. **Resilient Weather Retrieval**  
   - Implements **per-point caching** (`data/raw/openmeteo_cache/`), so repeated runs automatically reuse existing CSVs.  
   - Uses `requests.Session()` with **retry logic (exponential backoff)** to handle transient network errors.  
   - Applies a **delay between calls** to respect API rate limits.  
   - Normalizes outputs (lat/lon rounding, hourly alignment) for later joins with fire data.  

5. **Aggregation & Persistence**  
   - Concatenates all weather frames into a single DataFrame.  
   - Saves combined data under `data/raw/` with a date-stamped filename.  
   - Provides console feedback and previews the first rows for verification.  



In [ ]:
from time import sleep
from pathlib import Path
from io import StringIO
import json
import pandas as pd
import requests

# --- Preconditions
if "df_fires" not in globals() or df_fires is None or df_fires.empty:
    raise RuntimeError("df_fires is empty. Build it first (Cells 9→12).")

# --- Unique fire points (rounded to reduce duplicates)
pts = (
    df_fires[["latitude", "longitude"]]
    .dropna()
    .round(2)                     # 0.01° ~ 1.1 km
    .drop_duplicates()
    .reset_index(drop=True)
)
num_pts = len(pts)

# --- Safety cap and rate limiting
MAX_POINTS = 15                   # reduce a 15 para evitar timeouts
DELAY_SEC  = 0.8                  # pausa entre peticiones (ajustable)
RETRIES    = 3                    # reintentos por punto
TIMEOUT    = 60                   # timeout por petición (s)

if num_pts > MAX_POINTS:
    print(f"Found {num_pts} unique points; sampling first {MAX_POINTS} to limit API calls.")
    pts = pts.iloc[:MAX_POINTS].copy()
    num_pts = MAX_POINTS

# --- Cache dir per-point (resumable)
CACHE_DIR = RAW_DIR / "openmeteo_cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

session = requests.Session()

def fetch_point_with_retries(lat_i: float, lon_i: float) -> pd.DataFrame:
    """Call fetch_open_meteo with retries/backoff and return a normalized DataFrame."""
    backoff = 1.5
    err_last = None
    for attempt in range(1, RETRIES + 1):
        try:
            df_w = fetch_open_meteo(lat_i, lon_i, DATE_FROM, DATE_TO, model="era5")
            # Normalizar columnas y claves de join
            df_w = df_w.rename(columns={"lat": "latitude", "lon": "longitude"})
            df_w["lat_round"] = df_w["latitude"].round(1)
            df_w["lon_round"] = df_w["longitude"].round(1)
            df_w["time_hour"] = pd.to_datetime(df_w["time_utc"]).dt.floor("h")
            return df_w
        except Exception as e:
            err_last = e
            if attempt < RETRIES:
                sleep(backoff)
                backoff *= 2
            else:
                raise err_last

weather_frames = []

for i, row in pts.iterrows():
    lat_i = float(row["latitude"])
    lon_i = float(row["longitude"])

    # archivo de caché por punto (usa coordenadas redondeadas para nombre estable)
    key = f"{lat_i:.2f}_{lon_i:.2f}_{DATE_FROM}_{DATE_TO}.csv".replace(" ", "")
    cache_file = CACHE_DIR / key

    print(f"[{i+1}/{num_pts}] Weather lat={lat_i:.2f}, lon={lon_i:.2f} …", end=" ")
    if cache_file.exists():
        # usar caché
        df_w = pd.read_csv(cache_file, parse_dates=["time_utc", "time_hour"])
        print("(cached)")
    else:
        # llamar con reintentos y guardar caché
        df_w = fetch_point_with_retries(lat_i, lon_i)
        df_w.to_csv(cache_file, index=False)
        print("ok")
        sleep(DELAY_SEC)  # rate limiting

    weather_frames.append(df_w)

# --- Combine & save batch
if not weather_frames:
    raise RuntimeError("No weather frames were fetched. Check earlier steps.")

weather_points = pd.concat(weather_frames, ignore_index=True)
raw_weather_points_path = RAW_DIR / f"openmeteo_firepoints_{DATE_FROM}_{DATE_TO}.csv"
save_df(weather_points, raw_weather_points_path)

print(f" Weather fetched for {num_pts} fire point(s). Rows total: {len(weather_points)}")
display(weather_points.head(10))


## Cell 16 — Weather Join Validator (debug only)

This intermediate cell provides a **sanity check** before performing the full spatiotemporal join between FIRMS and Open-Meteo (Cell 16).  

**Purpose:**  
- Ensure that both datasets (`fires` and `weather_points`) contain the necessary harmonized keys for merging.  
- Validate that the temporal (`datetime_hour` / `time_hour`) and spatial (`lat_round`, `lon_round`) alignment columns are present.  
- Print row counts, preview available keys, and highlight potential gaps in coverage.  

**Why important:**  
Skipping this step can result in silent join failures (empty merges or missing columns), which would propagate errors downstream.  
By explicitly checking schemas and alignment, this validator cell reduces debugging time and improves pipeline robustness.  

**Note:**  
- This is a **diagnostic-only step**: it does not persist outputs.  
- Safe to skip in production once the pipeline has stabilized.  


In [ ]:
# --- Guards: ensure required dataframes exist -------------------------------
if "df_fires" not in globals() and "df_raw" not in globals():
    raise RuntimeError("No FIRMS dataframe found (df_fires or df_raw missing).")
if "weather_points" not in globals() or weather_points is None or weather_points.empty:
    raise RuntimeError("weather_points is empty. Re-run Cell 15 (batched Open-Meteo).")

# Choose the fire dataframe: prefer df_fires if available
fires_src = None
if "df_fires" in globals() and isinstance(df_fires, pd.DataFrame) and not df_fires.empty:
    fires_src = df_fires.copy()
elif "df_raw" in globals() and isinstance(df_raw, pd.DataFrame) and not df_raw.empty:
    fires_src = df_raw.copy()
else:
    raise RuntimeError("Both df_fires and df_raw are empty — cannot continue.")

# Normalize schemas for comparison
fires = fires_src.rename(columns=str.lower).copy()
wx    = weather_points.rename(columns=str.lower).copy()

# --- Check temporal alignment columns ---------------------------------------
print("=== Temporal alignment check ===")
print("FIRMS datetime columns:", [c for c in fires.columns if "date" in c or "time" in c])
print("Weather datetime columns:", [c for c in wx.columns if "time" in c])

# --- Check spatial alignment columns ----------------------------------------
print("\n=== Spatial alignment check ===")
print("FIRMS lat/lon columns:", [c for c in fires.columns if "lat" in c or "lon" in c])
print("Weather lat/lon columns:", [c for c in wx.columns if "lat" in c or "lon" in c])

# --- Row counts --------------------------------------------------------------
print("\n=== Row counts ===")
print("FIRMS rows:", len(fires))
print("Weather rows:", len(wx))

# --- Quick preview of keys ---------------------------------------------------
print("\n=== Preview of join keys ===")
if "acq_datetime" in fires.columns:
    fires["datetime_hour"] = pd.to_datetime(fires["acq_datetime"], utc=True, errors="coerce").dt.floor("h")
elif {"acq_date", "acq_time"}.issubset(fires.columns):
    t  = pd.Series(fires["acq_time"], dtype="string").str.zfill(4)
    dt = pd.Series(fires["acq_date"], dtype="string") + " " + t.str[:2] + ":" + t.str[2:] + ":00"
    fires["datetime_hour"] = pd.to_datetime(dt, utc=True, errors="coerce").dt.floor("h")

fires["lat_round"] = pd.to_numeric(fires.get("latitude", pd.Series()), errors="coerce").round(1)
fires["lon_round"] = pd.to_numeric(fires.get("longitude", pd.Series()), errors="coerce").round(1)

if "time_hour" not in wx.columns and "time_utc" in wx.columns:
    wx["time_hour"] = pd.to_datetime(wx["time_utc"]).dt.floor("h")
if "lat_round" not in wx.columns and "latitude" in wx.columns:
    wx["lat_round"] = pd.to_numeric(wx["latitude"], errors="coerce").round(1)
if "lon_round" not in wx.columns and "longitude" in wx.columns:
    wx["lon_round"] = pd.to_numeric(wx["longitude"], errors="coerce").round(1)

display(fires[["datetime_hour","lat_round","lon_round"]].head(5))
display(wx[["time_hour","lat_round","lon_round"]].head(5))

print("\n Validation complete — proceed to Cell 17 for the actual join.")


## Cell 17 — Join Coverage Diagnostics (place after Cell 16; before Cell 17)
Objective:
Quantify current merge coverage between FIRMS (df_fires) and Open-Meteo (weather_points) at two spatial roundings (±0.1°, ±0.2°). This is read-only: it does NOT modify existing artifacts.

In [ ]:
from typing import Tuple, Dict, List
import numpy as np
import pandas as pd

def _ensure_join_keys(
    fires_in: pd.DataFrame,
    wx_in: pd.DataFrame,
    round_deg: float
) -> Tuple[pd.DataFrame, pd.DataFrame, List[str]]:
    """Return (fires, wx) with normalized hourly time and spatial rounding for the join keys."""
    # Normalize schemas
    fires = fires_in.rename(columns=str.lower).copy()
    wx = wx_in.rename(columns=str.lower).copy()

    # --- Temporal alignment (UTC → hourly) ---
    # FIRMS
    if "acq_datetime" not in fires.columns:
        raise RuntimeError("df_fires lacks 'acq_datetime'. Run the cleaning cell first.")
    fires["acq_datetime"] = pd.to_datetime(fires["acq_datetime"], utc=True, errors="coerce")
    fires["datetime_hour"] = fires["acq_datetime"].dt.floor("h")

    # Weather
    if "time_hour" in wx.columns:
        wx["datetime_hour"] = pd.to_datetime(wx["time_hour"], utc=True, errors="coerce")
    elif "time_utc" in wx.columns:
        wx["datetime_hour"] = pd.to_datetime(wx["time_utc"], utc=True, errors="coerce").dt.floor("h")
    else:
        raise RuntimeError("weather_points lacks 'time_hour' or 'time_utc' columns.")

    # --- Spatial rounding (to specified grid) ---
    def _round_to_base(vals, base: float) -> np.ndarray:
        arr = np.array(pd.to_numeric(vals, errors="coerce"), dtype=float)
        return (np.round(arr / base) * base).astype(float)

    for col_src, col_dst in (("latitude", "lat_round"), ("longitude", "lon_round")):
        if col_src not in fires.columns:
            raise RuntimeError(f"df_fires lacks '{col_src}'.")
        fires[col_dst] = _round_to_base(fires[col_src], round_deg)

        if col_src not in wx.columns:
            raise RuntimeError(f"weather_points lacks '{col_src}'.")
        wx[col_dst] = _round_to_base(wx[col_src], round_deg)

    keys = ["datetime_hour", "lat_round", "lon_round"]
    return fires, wx, keys

def join_coverage(round_deg: float = 0.1) -> Dict[str, float]:
    """Compute coverage metrics for a given rounding tolerance."""
    assert "df_fires" in globals() and isinstance(df_fires, pd.DataFrame) and len(df_fires), \
        "df_fires is missing or empty."
    assert "weather_points" in globals() and isinstance(weather_points, pd.DataFrame) and len(weather_points), \
        "weather_points is missing or empty."

    fires, wx, keys = _ensure_join_keys(df_fires, weather_points, round_deg)

    # Unique key sets
    k_fires = fires[keys].dropna().drop_duplicates()
    k_wx = wx[keys].dropna().drop_duplicates()

    # Overlap of join keys
    k_inner = k_fires.merge(k_wx, on=keys, how="inner")
    coverage_keys = len(k_inner) / max(len(k_fires), 1)

    # Quick left-merge sample to probe temperature availability after join
    keep_cols_wx = [c for c in wx.columns if c in (
        "temperature_2m", "relative_humidity_2m", "windspeed_10m", "precipitation"
    )]
    probe = fires.merge(wx[keys + keep_cols_wx], on=keys, how="left", suffixes=("", "_wx"))

    # Temperature columns may appear under different names later; here we only test presence
    temp_like = [c for c in probe.columns if c.startswith("temperature_2m")]
    if temp_like:
        temp_any = pd.concat([pd.to_numeric(probe[c], errors="coerce") for c in temp_like], axis=1).notna().any(axis=1)
        temp_rows = int(temp_any.sum())
    else:
        temp_rows = 0

    return {
        "round_deg": round_deg,
        "fires_rows": int(len(fires)),
        "wx_rows": int(len(wx)),
        "fires_key_unique": int(len(k_fires)),
        "wx_key_unique": int(len(k_wx)),
        "key_overlap": int(len(k_inner)),
        "key_coverage_ratio": float(round(coverage_keys, 4)),
        "temperature_rows_after_left_merge": temp_rows,
        "temperature_ratio_vs_fires": float(round(temp_rows / max(len(fires), 1), 4)),
    }

# Run diagnostics at ±0.1° and ±0.2°
diag_01 = join_coverage(0.1)
diag_02 = join_coverage(0.2)

print("=== Join Coverage Diagnostics ===")
for d in (diag_01, diag_02):
    print(f"\nRounding: ±{d['round_deg']}°")
    print(f"- FIRMS rows: {d['fires_rows']} | unique join keys: {d['fires_key_unique']}")
    print(f"- Weather rows: {d['wx_rows']} | unique join keys: {d['wx_key_unique']}")
    print(f"- Key overlap: {d['key_overlap']}  (coverage={d['key_coverage_ratio']*100:.1f}%)")
    print(f"- Temperature presence after left-merge: {d['temperature_rows_after_left_merge']} "
          f"({d['temperature_ratio_vs_fires']*100:.1f}% of FIRMS rows)")


## Cell 17 — Join FIRMS and Open-Meteo Data

This cell performs the first **spatiotemporal integration** between satellite fire detections (NASA FIRMS) and local weather conditions (Open-Meteo).

**Operations**
1. **Schema harmonization**
   - Convert FIRMS timestamps to UTC and normalize to **hourly** resolution (`datetime_hour`).
   - Apply **soft spatial rounding** (0.1°) on lat/lon in both datasets to align with the meteorological grid.

2. **Join**
   - Merge on the composite keys: **(`datetime_hour`, `lat_round`, `lon_round`)**.
   - Produce a joined table with fire attributes and co-located hourly weather.

3. **Persistence**
   - Save the integrated dataset to `data/processed/` with a date-scoped filename.

4. **Diagnostics & Visualization**
   - Generate **Daily Fire Detections** figure from FIRMS.
   - Compute and plot **Daily Mean Temperature** at joined fire locations (if coverage is sufficient).
   - Store figures under `reports/` for traceability.

**Notes**
- This step assumes weather retrieval for fire points has been completed (see **Cell 14**).  
- Join quality depends on temporal alignment (hour rounding) and spatial tolerance (0.1°). These parameters can be tuned in subsequent iterations to maximize coverage without over-matching.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Preconditions ------------------------------------------------------
assert "df_fires" in globals() and len(df_fires), "df_fires missing."
assert "weather_points" in globals() and len(weather_points), "weather_points missing."
assert "PROCESSED_DIR" in globals() and "REPORTS_DIR" in globals(), "Run Global Configuration first."
assert "DATE_FROM" in globals() and "DATE_TO" in globals(), "Missing DATE_FROM/DATE_TO."

# --- Parameters ---------------------------------------------------------
TIME_TOL_H = 2        # temporal tolerance in hours (±2h) to boost coverage
MAX_RADIUS_KM = 40.0  # spatial tolerance (km)
EARTH_R_KM  = 6371.0088

# --- Normalize inputs ---------------------------------------------------
fires = df_fires.rename(columns=str.lower).copy()
wx    = weather_points.rename(columns=str.lower).copy()

fires["acq_datetime"]  = pd.to_datetime(fires["acq_datetime"], utc=True, errors="coerce")
fires["datetime_hour"] = fires["acq_datetime"].dt.floor("h")
fires = fires.dropna(subset=["latitude", "longitude", "datetime_hour"]).reset_index(drop=True)

if "datetime_hour" in wx.columns:
    wx["datetime_hour"] = pd.to_datetime(wx["datetime_hour"], utc=True, errors="coerce").dt.floor("h")
elif "time_hour" in wx.columns:
    wx["datetime_hour"] = pd.to_datetime(wx["time_hour"], utc=True, errors="coerce").dt.floor("h")
elif "time_utc" in wx.columns:
    wx["datetime_hour"] = pd.to_datetime(wx["time_utc"], utc=True, errors="coerce").dt.floor("h")
else:
    raise RuntimeError("weather_points lacks a datetime column (time_hour/time_utc).")

# Keep only features that exist in wx
wx_feats_all = [
    "temperature_2m",
    "relative_humidity_2m",
    "windspeed_10m",
    "precipitation",
    "dew_point_2m",
    "windgusts_10m",
    "winddirection_10m",
    "surface_pressure",
]
wx_feats = [c for c in wx_feats_all if c in wx.columns]
wx = wx[["datetime_hour","latitude","longitude"] + wx_feats].dropna(subset=["latitude","longitude","datetime_hour"])

# --- Expand temporal window (±2h) --------------------------------------
wx_expanded = []
for dt_shift in range(-TIME_TOL_H, TIME_TOL_H + 1):
    w = wx.copy()
    w["match_hour"] = w["datetime_hour"] + pd.Timedelta(hours=dt_shift)
    w["dt_offset_h"] = dt_shift
    wx_expanded.append(w)
wx_expanded = pd.concat(wx_expanded, ignore_index=True)

fires_ = fires.copy()
fires_["match_hour"] = fires_["datetime_hour"]

# --- Candidate join by hour, then pick nearest in space -----------------
cand = fires_[["acq_datetime","datetime_hour","latitude","longitude","match_hour"]].merge(
    wx_expanded,
    on="match_hour",
    how="left",
    suffixes=("_fire","_wx")
)

def haversine_km(lat1, lon1, lat2, lon2) -> np.ndarray:
    lat1 = np.radians(pd.to_numeric(lat1, errors="coerce").astype(float))
    lon1 = np.radians(pd.to_numeric(lon1, errors="coerce").astype(float))
    lat2 = np.radians(pd.to_numeric(lat2, errors="coerce").astype(float))
    lon2 = np.radians(pd.to_numeric(lon2, errors="coerce").astype(float))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2*np.arcsin(np.minimum(1.0, np.sqrt(a)))
    return EARTH_R_KM * c

cand["dist_km"] = haversine_km(
    cand["latitude"],  cand["longitude"],
    cand["latitude_wx"], cand["longitude_wx"]
)

cand = cand[cand["dist_km"] <= MAX_RADIUS_KM].copy()

# Resolve to the single best weather sample per fire row (min distance, then min |dt|)
cand["abs_dt_h"] = cand["dt_offset_h"].abs()
# Attach a stable fire_row id
fires_key = fires_[["acq_datetime","datetime_hour","latitude","longitude"]].reset_index().rename(columns={"index":"fire_row"})
cand = cand.merge(
    fires_key,
    on=["acq_datetime","datetime_hour","latitude","longitude"],
    how="left"
)
cand = cand.sort_values(["fire_row","dist_km","abs_dt_h"]).drop_duplicates(subset=["fire_row"], keep="first")

# --- Assemble final joined frame ---------------------------------------
keep_cols = [
    "fire_row","acq_datetime","datetime_hour","latitude","longitude",
    "datetime_hour_wx","dt_offset_h","latitude_wx","longitude_wx","dist_km"
] + wx_feats
df_join = cand.rename(columns={"datetime_hour_x":"datetime_hour"})  # just in case
df_join = df_join[keep_cols]

# --- Coverage metrics ---------------------------------------------------
def _nz(s): return int(pd.to_numeric(s, errors="coerce").notna().sum())
cov = {c: {"non_null": _nz(df_join[c])} for c in wx_feats}
for c in cov:
    cov[c]["ratio_vs_fires"] = float(round(cov[c]["non_null"] / max(len(fires_),1), 4))

print(f"FIRMS rows: {len(fires_)} | WX rows: {len(wx)} | Candidates kept: {len(df_join)}")
print(f"Radius ≤ {MAX_RADIUS_KM:.0f} km, time window ±{TIME_TOL_H}h")
print("\n=== Coverage after NN-join ===")
for k,v in cov.items():
    print(f"- {k:>20s}: {v['non_null']:6d} rows  ({v['ratio_vs_fires']*100:5.1f}% of FIRMS rows)")

# --- Persist ------------------------------------------------------------
out_csv = PROCESSED_DIR / f"fires_weather_es_{DATE_FROM}_{DATE_TO}_nn_{int(MAX_RADIUS_KM)}km_pm{TIME_TOL_H}h.csv"
df_join.to_csv(out_csv, index=False)
print("\nSaved NN-joined CSV →", out_csv)

# --- Safe plotting helpers ---------------------------------------------
def _save_line_safe(df_xy: pd.DataFrame, xcol: str, ycol: str, title: str, ylabel: str, fname: str):
    # Emit plot only if there is valid data
    if ycol not in df_xy.columns or pd.to_numeric(df_xy[ycol], errors="coerce").notna().sum() == 0:
        print(f"Skipped plot (no data): {title}")
        return
    plt.figure(figsize=(8.2, 4.4))
    plt.plot(df_xy[xcol], df_xy[ycol])
    plt.title(title)
    plt.xlabel("Date (UTC)")
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    outp = REPORTS_DIR / fname
    plt.savefig(outp, dpi=160)
    plt.show()
    print("Saved:", outp)

# --- Figures (only when data exists) -----------------------------------
# 0) Daily detections (always exists)
daily_det = (
    fires_.assign(day=fires_["acq_datetime"].dt.floor("D"))
    .groupby("day")["acq_datetime"].count()
    .reset_index(name="detections")
)
_save_line_safe(
    daily_det, "day", "detections",
    "Daily Fire Detections (FIRMS)",
    "Detections",
    f"nn_daily_detections_{DATE_FROM}_{DATE_TO}.png"
)

# 1) Daily meteo means on matched rows (skip if empty)
if "datetime_hour" in df_join.columns:
    base = df_join.assign(day=pd.to_datetime(df_join["datetime_hour"], utc=True).dt.floor("D"))
    if "temperature_2m" in df_join.columns:
        tmp = base.groupby("day")["temperature_2m"].mean().reset_index(name="temp_mean")
        _save_line_safe(tmp, "day", "temp_mean",
                        "Daily Mean Temperature (NN-Joined)",
                        "Temp 2m (°C)",
                        f"nn_daily_temp_{DATE_FROM}_{DATE_TO}.png")
    if "relative_humidity_2m" in df_join.columns:
        tmp = base.groupby("day")["relative_humidity_2m"].mean().reset_index(name="rh_mean")
        _save_line_safe(tmp, "day", "rh_mean",
                        "Daily Mean Relative Humidity (NN-Joined)",
                        "RH 2m (%)",
                        f"nn_daily_rh_{DATE_FROM}_{DATE_TO}.png")
    if "windspeed_10m" in df_join.columns:
        tmp = base.groupby("day")["windspeed_10m"].mean().reset_index(name="wind_mean")
        _save_line_safe(tmp, "day", "wind_mean",
                        "Daily Mean Wind Speed (NN-Joined)",
                        "Wind 10m (m/s)",
                        f"nn_daily_wind_{DATE_FROM}_{DATE_TO}.png")
    if "precipitation" in df_join.columns:
        tmp = base.groupby("day")["precipitation"].sum().reset_index(name="precip_sum")
        _save_line_safe(tmp, "day", "precip_sum",
                        "Daily Precipitation Sum (NN-Joined)",
                        "Precipitation (mm)",
                        f"nn_daily_precip_{DATE_FROM}_{DATE_TO}.png")


In [ ]:
import pandas as pd

def save_df(df: pd.DataFrame, path: Path) -> None:
    """Save a DataFrame to CSV and print the path."""
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False)
    print(f"Saved: {path}")

def preview(df: pd.DataFrame, n: int = MAX_ROWS_PREVIEW) -> None:
    """Quick preview of a DataFrame."""
    display(df.head(n))

